In [ ]:
# YOLOv8 — Dataset Check → (optional) Auto-fix → Visualize → Train → Validate (STOP HERE)

from pathlib import Path

# ====== EDIT THESE ======
DATASET_DIR = Path(r"/home/surya/Downloads/soccer players.v1i.yolov8")  # your extracted dataset folder
PROJECT_NAME  = "soccer_players_clean"
IMG_SIZE      = 640       # try 960/1280 for small objects (applies to both width and height)
EPOCHS        = 80        # raise if dataset is small (100–150)
BATCH         = 4
MODEL_WEIGHTS = "yolov8n.pt"  # try yolov8s.pt later
DO_AUTOFIX    = True      # set False to only report issues
N_VIS_SAMPLES = 24        # annotated preview images per split
# ========================

ROOT       = Path.cwd()
WORK_DIR   = ROOT / "datasets" / PROJECT_NAME
REPORT_DIR = ROOT / "dataset_report"

WORK_DIR, REPORT_DIR
WORK_DIR.mkdir(parents=True, exist_ok=True)
REPORT_DIR.mkdir(parents=True, exist_ok=True)
IMG_DIRS = {
    "train": DATASET_DIR / "images" / "train",
    "val":   DATASET_DIR / "images" / "val",
    "test":  DATASET_DIR / "images" / "test",
}
LABEL_DIRS = {
    "train": DATASET_DIR / "labels" / "train",
    "val":   DATASET_DIR / "labels" / "val",
    "test":  DATASET_DIR / "labels" / "test",
}


In [3]:
%pip install ultralytics supervision matplotlib pandas seaborn opencv-python-headless
import sys, platform, torch, subprocess, shlex

print("Python:", sys.version)
print("Platform:", platform.platform())
print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("CUDA device:", torch.cuda.get_device_name(0))
    print("Compute capability:", torch.cuda.get_device_capability(0))
    print("Total VRAM (GB):", round(torch.cuda.get_device_properties(0).total_memory/1e9,2))
try:
    print(subprocess.check_output(shlex.split("nvidia-smi"), text=True))
except Exception as e:
    print("nvidia-smi not available or failed:", e)


Note: you may need to restart the kernel to use updated packages.
Python: 3.12.3 (main, Jun 18 2025, 17:59:45) [GCC 13.3.0]
Platform: Linux-6.14.0-28-generic-x86_64-with-glibc2.39
Torch: 2.8.0+cu128
CUDA available: False
nvidia-smi not available or failed: [Errno 2] No such file or directory: 'nvidia-smi'


In [5]:
import shutil, yaml, os
from pathlib import Path

def mirror_dataset(src: Path, dst_root: Path) -> Path:
    assert src.is_dir(), f"Dataset folder not found: {src}"
    dst = dst_root / src.name
    dst.parent.mkdir(parents=True, exist_ok=True)
    if not dst.exists() or not any(dst.iterdir()):
        print(f"Copying {src} -> {dst}")
        if dst.exists():
            shutil.rmtree(dst)
        shutil.copytree(src, dst)
    else:
        print(f"Using existing mirrored dataset: {dst}")
    return dst

def find_data_yaml(base: Path):
    cand = base / "data.yaml"
    if cand.exists():
        return cand
    for sd in [p for p in base.iterdir() if p.is_dir()]:
        c = sd / "data.yaml"
        if c.exists():
            return c
    allc = list(base.rglob("data.yaml"))
    if allc:
        allc.sort(key=lambda p: len(p.parts))
        return allc[0]
    return None

DATA_ROOT = mirror_dataset(DATASET_DIR, WORK_DIR.parent)
DATA_YAML = find_data_yaml(DATA_ROOT)
assert DATA_YAML and DATA_YAML.exists(), f"data.yaml not found under {DATA_ROOT}"
print("DATA_ROOT:", DATA_ROOT)
print("data.yaml:", DATA_YAML)

with open(DATA_YAML, "r") as f:
    spec = yaml.safe_load(f)

base = Path(spec.get("path", DATA_YAML.parent))

def resolve_dir(key, default):
    p = spec.get(key, default)
    p = Path(p)
    if not p.is_absolute():
        p = base / p
    return p

val_key = "val" if "val" in spec else ("valid" if "valid" in spec else "val")
train_images = resolve_dir("train", "train/images")
val_images   = resolve_dir(val_key, f"{val_key}/images")
test_images  = resolve_dir("test",  "test/images")

print("train images:", train_images, "exists:", train_images.exists())
print("val images:  ", val_images,   "exists:", val_images.exists())
print("test images: ", test_images,  "exists:", test_images.exists())


Using existing mirrored dataset: /home/surya/c/datasets/soccer players.v1i.yolov8
DATA_ROOT: /home/surya/c/datasets/soccer players.v1i.yolov8
data.yaml: /home/surya/c/datasets/soccer players.v1i.yolov8/data.yaml
train images: /home/surya/c/datasets/soccer players.v1i.yolov8/../train/images exists: False
val images:   /home/surya/c/datasets/soccer players.v1i.yolov8/../valid/images exists: False
test images:  /home/surya/c/datasets/soccer players.v1i.yolov8/../test/images exists: False


In [6]:
from collections import Counter
import pandas as pd
from pathlib import Path

REPORT_DIR.mkdir(parents=True, exist_ok=True)

def load_classes(spec):
    names = spec.get("names", [])
    if isinstance(names, dict):
        names = [names[i] for i in sorted(map(int, names.keys()))]
    return names, len(names)

CLASS_NAMES, NUM_CLASSES = load_classes(spec)
print("Classes:", CLASS_NAMES, " (count:", NUM_CLASSES, ")")

def image_label_pairs(split_images: Path):
    pairs = []
    for img in split_images.rglob("*"):
        if img.suffix.lower() in {".jpg",".jpeg",".png",".bmp",".tif",".tiff"}:
            rel = img.relative_to(split_images)
            lbl = split_images.parent / "labels" / rel.with_suffix(".txt")
            pairs.append((img, lbl))
    return pairs

def parse_label_file(lbl_path: Path):
    lines = []
    if not lbl_path.exists():
        return lines
    for i, line in enumerate(lbl_path.read_text().splitlines(), 1):
        line=line.strip()
        if not line:
            continue
        parts = line.split()
        if len(parts) != 5:
            lines.append(("bad_format", i, line)); continue
        try:
            cid = int(float(parts[0])); x,y,w,h = map(float, parts[1:])
        except Exception:
            lines.append(("parse_error", i, line)); continue
        lines.append(("ok", i, (cid,x,y,w,h)))
    return lines

def validate_split(split_name, split_images):
    pairs = image_label_pairs(split_images)
    issues = []
    class_counts = Counter()
    tiny_counts = Counter()
    for img, lbl in pairs:
        if not lbl.exists():
            issues.append((split_name, "missing_label", str(img), "")); continue
        parsed = parse_label_file(lbl)
        if not parsed:
            continue
        for status, ln, payload in parsed:
            if status != "ok":
                issues.append((split_name, status, str(lbl), f"line {ln}: {payload}")); continue
            cid,x,y,w,h = payload
            if cid < 0 or cid >= NUM_CLASSES:
                issues.append((split_name, "bad_class_id", str(lbl), f"line {ln}: {cid} not in [0,{NUM_CLASSES-1}]")); continue
            if not (0.0 <= x <= 1.0 and 0.0 <= y <= 1.0 and 0.0 < w <= 1.0 and 0.0 < h <= 1.0):
                issues.append((split_name, "out_of_range", str(lbl), f"line {ln}: {x},{y},{w},{h}")); continue
            area = w*h
            if area < 0.005:   # <0.5% of image area (tiny)
                tiny_counts[cid]+=1
                issues.append((split_name, "tiny_box_warn", str(lbl), f"line {ln}: area={area:.5f}"))
            class_counts[cid]+=1

    # labels without images
    labels_path = split_images.parent / "labels"
    for lbl in labels_path.rglob("*.txt"):
        rel = lbl.relative_to(labels_path)
        img = split_images / rel.with_suffix(".jpg")
        if not img.exists():
            found=False
            for ext in [".jpeg",".png",".bmp",".tif",".tiff",".JPG",".PNG"]:
                if (split_images / rel.with_suffix(ext)).exists():
                    found=True; break
            if not found:
                issues.append((split_name, "missing_image_for_label", str(lbl), ""))

    return pairs, issues, class_counts, tiny_counts

all_issues = []
split_stats = {}
for split_name, folder in [("train", train_images), (val_key, val_images), ("test", test_images)]:
    if folder.exists():
        pairs, issues, class_counts, tiny_counts = validate_split(split_name, folder)
        split_stats[split_name] = {"images": len(pairs), "class_counts": dict(class_counts), "tiny_counts": dict(tiny_counts)}
        all_issues.extend(issues)

issues_df = pd.DataFrame(all_issues, columns=["split","issue","path","detail"]) if all_issues else pd.DataFrame(columns=["split","issue","path","detail"])
issues_csv = REPORT_DIR / "dataset_issues.csv"
issues_df.to_csv(issues_csv, index=False)
print(f"Issues found: {len(issues_df)}  -> {issues_csv}")
split_stats


Classes: ['0', '1', '2']  (count: 3 )
Issues found: 0  -> /home/surya/c/dataset_report/dataset_issues.csv


{}

In [7]:
import pandas as pd
import random, cv2, numpy as np
from pathlib import Path

def summarize_distribution(split_stats, class_names):
    rows = []
    for split, st in split_stats.items():
        row = {"split": split, "images": st["images"]}
        for i, cname in enumerate(class_names):
            row[f"class_{i}:{cname}"] = st["class_counts"].get(i, 0)
        rows.append(row)
    return pd.DataFrame(rows)

dist_df = summarize_distribution(split_stats, CLASS_NAMES)
dist_csv = REPORT_DIR / "class_distribution.csv"
dist_df.to_csv(dist_csv, index=False)
print("Saved class distribution ->", dist_csv)
display(dist_df) if 'display' in globals() else print(dist_df)

def image_label_pairs(split_images: Path):
    pairs = []
    for img in split_images.rglob("*"):
        if img.suffix.lower() in {".jpg",".jpeg",".png",".bmp",".tif",".tiff"}:
            rel = img.relative_to(split_images)
            lbl = split_images.parent / "labels" / rel.with_suffix(".txt")
            pairs.append((img, lbl))
    return pairs

def draw_yolo_boxes(img_path: Path, lbl_path: Path, names):
    img = cv2.imread(str(img_path))
    if img is None:
        return None
    h, w = img.shape[:2]
    colors = {i: ((37*i)%255, (17*i)%255, (97*i)%255) for i in range(len(names))}
    if lbl_path.exists():
        for line in lbl_path.read_text().splitlines():
            parts = line.strip().split()
            if len(parts) != 5: 
                continue
            try:
                cid = int(float(parts[0])); xc, yc, bw, bh = map(float, parts[1:])
            except:
                continue
            x1 = int((xc - bw/2) * w); y1 = int((yc - bh/2) * h)
            ww = int(bw * w); hh = int(bh * h)
            x1 = max(0, x1); y1 = max(0, y1); ww = max(1, ww); hh = max(1, hh)
            cv2.rectangle(img, (x1,y1), (x1+ww,y1+hh), colors.get(cid,(0,255,0)), 2)
            cv2.putText(img, names[cid] if cid<len(names) else str(cid), (x1,y1-5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, colors.get(cid,(0,255,0)), 2)
    return img

def save_previews(split_name, split_images, n=24):
    out_dir = REPORT_DIR / "previews" / split_name
    out_dir.mkdir(parents=True, exist_ok=True)
    pairs = image_label_pairs(split_images)
    random.seed(0); random.shuffle(pairs)
    saved = 0
    for img, lbl in pairs:
        vis = draw_yolo_boxes(img, lbl, CLASS_NAMES)
        if vis is None: 
            continue
        out = out_dir / f"{img.stem}_preview.jpg"
        cv2.imwrite(str(out), vis)
        saved += 1
        if saved >= n:
            break
    print(f"Saved {saved} previews -> {out_dir}")

for split_name, folder in [("train", train_images), (val_key, val_images), ("test", test_images)]:
    if folder.exists():
        save_previews(split_name, folder, N_VIS_SAMPLES)


Saved class distribution -> /home/surya/c/dataset_report/class_distribution.csv
Empty DataFrame
Columns: []
Index: []


In [8]:
import pandas as pd

def autofix_labels(split_images, drop_empty=False):
    fixed, dropped = 0, 0
    labels_path = split_images.parent / "labels"
    for lbl in labels_path.rglob("*.txt"):
        lines_in = lbl.read_text().splitlines() if lbl.exists() else []
        new_lines, changed = [], False
        for line in lines_in:
            parts = line.strip().split()
            if len(parts) != 5:
                dropped += 1; changed=True; continue
            try:
                cid = int(float(parts[0])); xc, yc, bw, bh = map(float, parts[1:])
            except:
                dropped += 1; changed=True; continue
            if cid < 0 or cid >= NUM_CLASSES:
                dropped += 1; changed=True; continue
            # clip to [0,1], avoid zero size
            orig = (xc,yc,bw,bh)
            xc = min(max(xc, 0.0), 1.0)
            yc = min(max(yc, 0.0), 1.0)
            bw = min(max(bw, 1e-6), 1.0)
            bh = min(max(bh, 1e-6), 1.0)
            if bw <= 0 or bh <= 0:
                dropped += 1; changed=True; continue
            if orig != (xc,yc,bw,bh):
                changed = True
            new_lines.append(f"{cid} {xc:.6f} {yc:.6f} {bw:.6f} {bh:.6f}")
        if drop_empty and not new_lines:
            lbl.unlink(missing_ok=True); changed=True
        if changed:
            lbl.write_text("\n".join(new_lines)); fixed += 1
    return fixed, dropped

if DO_AUTOFIX:
    for split_name, folder in [("train", train_images), (val_key, val_images), ("test", test_images)]:
        if folder.exists():
            fixed, dropped = autofix_labels(folder, drop_empty=False)
            print(f"[{split_name}] files fixed: {fixed}, lines dropped: {dropped}")

# quick re-validate
all_issues2 = []
split_stats2 = {}
for split_name, folder in [("train", train_images), (val_key, val_images), ("test", test_images)]:
    if folder.exists():
        pairs, issues, class_counts, tiny_counts = validate_split(split_name, folder)
        split_stats2[split_name] = {"images": len(pairs), "class_counts": dict(class_counts), "tiny_counts": dict(tiny_counts)}
        all_issues2.extend(issues)

issues2_df = pd.DataFrame(all_issues2, columns=["split","issue","path","detail"]) if all_issues2 else pd.DataFrame(columns=["split","issue","path","detail"])
issues2_csv = REPORT_DIR / "dataset_issues_postfix.csv"
issues2_df.to_csv(issues2_csv, index=False)
print(f"Post-fix issues: {len(issues2_df)}  -> {issues2_csv}")
split_stats2


Post-fix issues: 0  -> /home/surya/c/dataset_report/dataset_issues_postfix.csv


{}

In [9]:
import sys
!{sys.executable} -m pip -q install ultralytics

from ultralytics import YOLO
import torch

use_gpu = torch.cuda.is_available()
print("GPU available:", use_gpu)

model = YOLO(MODEL_WEIGHTS)

# (Optional) 1-epoch smoke test on CPU:
# model.train(data=str(DATA_YAML), imgsz=320, batch=2, epochs=1, device="cpu", workers=0, name=f"{PROJECT_NAME}-smoke_cpu")

# Proper training run
res = model.train(
    data=str(DATA_YAML),
    imgsz=IMG_SIZE,
    batch=BATCH,
    epochs=EPOCHS,
    device=0 if use_gpu else "cpu",
    workers=0,     # notebook-safe
    amp=True,      # mixed precision
    cache=False,   # enable only if you have lots of RAM
    project="runs/train",
    name=f"{PROJECT_NAME}-exp",
)
print("Training done.")


GPU available: False
Ultralytics 8.3.185 🚀 Python-3.12.3 torch-2.8.0+cu128 CPU (11th Gen Intel Core(TM) i5-11300H 3.10GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/home/surya/c/datasets/soccer players.v1i.yolov8/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=80, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=soccer_players_clean-exp2, nbs=64, nms=False, opset=None, optimize=False, optimizer=a

train: Scanning /home/surya/c/datasets/soccer players.v1i.yolov8/train/labels.cache... 114 images, 0 backgrounds, 0 corrupt: 100%|██████████| 114/114 [00:00<?, ?it/s]

val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 145.7±87.5 MB/s, size: 32.3 KB)



/home/surya/c/.venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
val: Scanning /home/surya/c/datasets/soccer players.v1i.yolov8/valid/labels.cache... 33 images, 0 backgrounds, 0 corrupt: 100%|██████████| 33/33 [00:00<?, ?it/s]
/home/surya/c/.venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Plotting labels to runs/train/soccer_players_clean-exp2/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001429, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs/train/soccer_players_clean-exp2
Starting training for 80 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      80/80         0G      1.054     0.5633     0.9038         21        640: 100%|██████████| 29/29 [01:01<00:00,  2.11s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:03<00:00,  1.35it/s]

                   all         33        407      0.949      0.867      0.911      0.561



80 epochs completed in 1.662 hours.
Optimizer stripped from runs/train/soccer_players_clean-exp2/weights/last.pt, 6.2MB
Optimizer stripped from runs/train/soccer_players_clean-exp2/weights/best.pt, 6.2MB

Validating runs/train/soccer_players_clean-exp2/weights/best.pt...
Ultralytics 8.3.185 🚀 Python-3.12.3 torch-2.8.0+cu128 CPU (11th Gen Intel Core(TM) i5-11300H 3.10GHz)
Model summary (fused): 72 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:03<00:00,  1.41it/s]


                   all         33        407      0.965      0.869      0.918      0.569
                     0         33        359       0.99      0.978      0.991      0.649
                     1         20         20      0.962       0.95      0.976      0.714
                     2         28         28      0.942      0.679      0.787      0.345
Speed: 3.1ms preprocess, 92.9ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs/train/soccer_players_clean-exp2
Training done.


In [ ]:
from ultralytics import YOLO
model = YOLO("runs/train/soccer_players_clean-exp/weights/best.pt")

onnx_path = model.export(format="onnx", imgsz=IMG_SIZE, opset=12)
print("Exported ONNX:", onnx_path)


Ultralytics 8.3.185 🚀 Python-3.12.3 torch-2.8.0+cu128 CPU (11th Gen Intel Core(TM) i5-11300H 3.10GHz)
Model summary (fused): 72 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from 'runs/train/soccer_players_clean-exp/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 7, 8400) (17.6 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0,<1.18.0', 'onnxslim>=0.1.59', 'onnxruntime'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 5.2 MB/s  0:00:03m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 6.9 MB/s  0:00:02 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8/8 [onnxruntime] [onnxruntime]

requirements: AutoUpdate success ✅ 14.8s
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.17.0 opset 12...
ONNX: slimming with onnxslim 0.1.65...
ONNX: export success ✅ 17.3s, saved as 'runs/train/so

In [14]:
from pathlib import Path

onnx_file = Path(onnx_path)
if onnx_file.exists():
    print("ONNX file exported successfully:", onnx_file.resolve())
else:
    print("ONNX export failed — file not found.")


ONNX file exported successfully: /home/surya/c/runs/train/soccer_players_clean-exp/weights/best.onnx


In [15]:
from pathlib import Path
import shutil, random

HAILO_DIR = Path.cwd() / "hailo"
CALIB_DIR = HAILO_DIR / "calib"
CALIB_DIR.mkdir(parents=True, exist_ok=True)

exts = {".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff"}
all_imgs = [p for p in train_images.rglob("*") if p.suffix.lower() in exts]
random.seed(42); random.shuffle(all_imgs)

# Choose up to 1000 diverse images for calibration
pick = all_imgs[:1000] if len(all_imgs) > 1000 else all_imgs
for p in pick:
    shutil.copy2(p, CALIB_DIR / p.name)

print(f"Calibration images prepared: {len(pick)} at {CALIB_DIR}")


Calibration images prepared: 0 at /home/surya/c/hailo/calib


In [16]:
from pathlib import Path

print("train_images:", train_images, "exists:", train_images.exists())
print("val_images  :", val_images,   "exists:", val_images.exists())
print("test_images :", test_images,  "exists:", test_images.exists())

# Count by split and show a few examples
def count_and_sample(folder: Path, max_show=10):
    if not folder or not folder.exists():
        return 0, []
    exts = {".jpg",".jpeg",".png",".bmp",".tif",".tiff",".JPG",".PNG",".JPEG",".BMP",".TIF",".TIFF"}
    imgs = [p for p in folder.rglob("*") if p.suffix in exts]
    print(f"{folder} -> {len(imgs)} images")
    for p in imgs[:max_show]:
        print("  ", p)
    return len(imgs), imgs[:max_show]

n_train, _ = count_and_sample(train_images)
n_val,   _ = count_and_sample(val_images)
n_test,  _ = count_and_sample(test_images)

# Fallback: scan the whole dataset root (in case YAML paths are odd)
DATA_ROOT
all_any = [p for p in DATA_ROOT.rglob("*") if p.suffix.lower() in {".jpg",".jpeg",".png",".bmp",".tif",".tiff"}]
print(f"All images under DATA_ROOT: {len(all_any)} (first 10 shown)")
for p in all_any[:10]:
    print("  ", p)


train_images: /home/surya/c/datasets/soccer players.v1i.yolov8/../train/images exists: False
val_images  : /home/surya/c/datasets/soccer players.v1i.yolov8/../valid/images exists: False
test_images : /home/surya/c/datasets/soccer players.v1i.yolov8/../test/images exists: False
All images under DATA_ROOT: 163 (first 10 shown)
   /home/surya/c/datasets/soccer players.v1i.yolov8/train/images/1-fps-2_00278_jpeg_jpg.rf.076b339a54a17d03949edac9cee5e9eb.jpg
   /home/surya/c/datasets/soccer players.v1i.yolov8/train/images/1-fps-2_00356_jpeg_jpg.rf.fdc37ae2fd1a1854d7740feb20a72b96.jpg
   /home/surya/c/datasets/soccer players.v1i.yolov8/train/images/1-fps-2_00009_jpeg_jpg.rf.4e13eea75a7acdd1eb847744fdff39c4.jpg
   /home/surya/c/datasets/soccer players.v1i.yolov8/train/images/1-fps-2_00718_jpeg_jpg.rf.d3780ffd86a6d5db62974943c65d43ac.jpg
   /home/surya/c/datasets/soccer players.v1i.yolov8/train/images/1-fps-2_00038_jpeg_jpg.rf.c3b4d342523fe527e69b9c11b579c184.jpg
   /home/surya/c/datasets/soccer 

In [17]:
from pathlib import Path
import shutil, random

HAILO_DIR = Path.cwd() / "hailo"
CALIB_DIR = HAILO_DIR / "calib"
CALIB_DIR.mkdir(parents=True, exist_ok=True)

# Collect from train, val, test in that order; if none, fall back to any under DATA_ROOT
exts = {".jpg",".jpeg",".png",".bmp",".tif",".tiff",".JPG",".PNG",".JPEG",".BMP",".TIF",".TIFF"}

def gather(folder: Path):
    return [p for p in folder.rglob("*") if p.suffix in exts] if folder and folder.exists() else []

pool = []
for folder in [train_images, val_images, test_images]:
    pool.extend(gather(folder))

if not pool:
    print("No split images found; falling back to scan entire DATA_ROOT.")
    pool = [p for p in DATA_ROOT.rglob("*") if p.suffix.lower() in {".jpg",".jpeg",".png",".bmp",".tif",".tiff"}]

# Deduplicate by name (to avoid overwrites); if duplicates, append an index
random.seed(42)
random.shuffle(pool)

# Cap at ~1000 images for calibration (use fewer if dataset is small)
cap = min(1000, len(pool))
picked = pool[:cap]

# Ensure unique filenames in CALIB_DIR
used = set()
copied = 0
for i, src in enumerate(picked):
    base = src.name
    name = base
    stem, suf = src.stem, src.suffix
    k = 1
    while name in used or (CALIB_DIR / name).exists():
        name = f"{stem}_{k}{suf}"
        k += 1
    used.add(name)
    shutil.copy2(src, CALIB_DIR / name)
    copied += 1

print(f"Calibration images prepared: {copied} at {CALIB_DIR}")
if copied == 0:
    print("⚠️ Still zero. Check that DATA_ROOT is correct and that images actually exist on disk.")


No split images found; falling back to scan entire DATA_ROOT.
Calibration images prepared: 163 at /home/surya/c/hailo/calib


In [10]:
# Webcam real-time tester for YOLOv8 (with live controls, FPS, recording)
# Keys:
#   q  = quit (releases camera cleanly)
#   -  = decrease confidence threshold (e.g., 0.50 → 0.40 → 0.30 …)
#   =  = increase confidence threshold
#   t  = toggle TTA (test-time augmentation)  [off by default; costs FPS]
#   r  = start/stop recording annotated video to ./runs/webcam/
#   p  = pause/resume
#
# Tips for screen-capture scenarios: use higher imgsz (960/1280), lower conf (0.10–0.35),
# darken room to reduce glare, square the webcam to the display to avoid perspective skew.

import cv2, time, os
from pathlib import Path
from datetime import datetime
from ultralytics import YOLO

# ==== EDIT THESE ====
MODEL_PATH = "runs/train/soccer_players_clean-exp/weights/best.pt"
CAM_INDEX  = 0            # default webcam
IMG_SIZE   = 960          # 640 ok, but 960/1280 helps small players/ball
CONF       = 0.25         # start stricter; lower live with '-' key if nothing appears
IOU        = 0.60
USE_TTA    = False        # toggle live with 't'
DISPLAY_W, DISPLAY_H = None, None  # set e.g. (1280, 720) to force viewer size
# =====================

# Load model once
model = YOLO(MODEL_PATH)

# Open webcam
cap = cv2.VideoCapture(CAM_INDEX)
if not cap.isOpened():
    raise RuntimeError(f"❌ Could not open webcam index {CAM_INDEX}")

# (Optional) try to set higher capture resolution for cleaner frames
try:
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
    cap.set(cv2.CAP_PROP_FPS, 30)
except Exception:
    pass

# Recording
save_root = Path("runs/webcam")
save_root.mkdir(parents=True, exist_ok=True)
writer = None
recording = False

# FPS smoothing
t0 = time.time()
frame_count = 0
fps_ema = None
paused = False

def start_writer(frame, root):
    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    out_path = root / f"webcam_{ts}.mp4"
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    h, w = frame.shape[:2]
    w_out, h_out = (DISPLAY_W or w), (DISPLAY_H or h)
    vw = cv2.VideoWriter(str(out_path), fourcc, 30.0, (w_out, h_out))
    return vw, out_path

print("✅ Webcam open. Press 'q' to quit.  Live controls: -, =, t, r, p")

try:
    while True:
        if not paused:
            ok, frame = cap.read()
            if not ok:
                print("⚠️ Frame grab failed; stopping.")
                break

            # Inference (Ultralytics handles letterboxing internally)
            # Use direct call `model(frame, ...)` for lower overhead than .predict on streams
                annotated = results[0].plot()  # draw boxes, labels, conf

            # FPS
            frame_count += 1
            if frame_count >= 1:
                dt = time.time() - t0
                inst_fps = frame_count / max(dt, 1e-9)
                fps_ema = inst_fps if fps_ema is None else (0.9 * fps_ema + 0.1 * inst_fps)

            # Overlay status text
            overlay = annotated
            h, w = overlay.shape[:2]
            if DISPLAY_W and DISPLAY_H:
                overlay = cv2.resize(overlay, (DISPLAY_W, DISPLAY_H))

            hud = f"FPS: {fps_ema:.1f} | imgsz: {IMG_SIZE} | conf: {CONF:.2f} | iou: {IOU:.2f} | TTA: {USE_TTA} | rec: {recording}"
            cv2.putText(overlay, hud, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0), 2, cv2.LINE_AA)
        else:
            overlay = cv2.putText(overlay, "PAUSED (press 'p' to resume)", (10, 60),
                                  cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,255), 2, cv2.LINE_AA)

        # Show
        cv2.imshow("YOLOv8 Webcam (press q to quit)", overlay)

        # Write video if recording
        if recording:
            if writer is None:
                writer, current_out = start_writer(overlay, save_root)
                print(f"⏺️ Recording to: {current_out}")
            writer.write(overlay)
        elif writer is not None:
            writer.release()
            print("⏹️ Recording stopped.")
            writer = None

        # Keys
        k = cv2.waitKey(1) & 0xFF
        if k == ord('q'):
            break
        elif k == ord('-'):
            CONF = max(0.05, CONF - 0.05)
        elif k == ord('=') or k == ord('+'):
            CONF = min(0.95, CONF + 0.05)
        elif k == ord('t'):
            USE_TTA = not USE_TTA
        elif k == ord('r'):
            recording = not recording
        elif k == ord('p'):
            paused = not paused
        elif k == ord('['):
            IMG_SIZE = max(320, IMG_SIZE - 64)
        elif k == ord(']'):
            IMG_SIZE = min(1536, IMG_SIZE + 64)

finally:
    # Clean release
    if writer is not None:
        writer.release()
    cap.release()
    cv2.destroyAllWindows()
    print("✅ Webcam released cleanly")


✅ Webcam open. Press 'q' to quit.  Live controls: -, =, t, r, p
✅ Webcam released cleanly


In [31]:
import sys, os, shutil, subprocess, stat
!{sys.executable} -m pip -q install yt-dlp imageio-ffmpeg ultralytics opencv-python

# Get a real ffmpeg binary (bundled) and ensure it's executable, then add to PATH
import imageio_ffmpeg
ffmpeg_path = imageio_ffmpeg.get_ffmpeg_exe()

# Make executable (Linux/macOS safety)
try:
    os.chmod(ffmpeg_path, os.stat(ffmpeg_path).st_mode | stat.S_IEXEC)
except Exception as e:
    print("chmod skipped:", e)

ffmpeg_dir = os.path.dirname(ffmpeg_path)
os.environ["PATH"] = ffmpeg_dir + os.pathsep + os.environ.get("PATH", "")

# Sanity: show ffmpeg version from this binary
print("Using FFmpeg at:", ffmpeg_path)
print(subprocess.check_output([ffmpeg_path, "-version"], text=True).splitlines()[0])


Using FFmpeg at: /home/surya/c/.venv/lib/python3.12/site-packages/imageio_ffmpeg/binaries/ffmpeg-linux-x86_64-v7.0.2
ffmpeg version 7.0.2-static https://johnvansickle.com/ffmpeg/  Copyright (c) 2000-2024 the FFmpeg developers


In [34]:
import yt_dlp
import cv2
from ultralytics import YOLO
from pathlib import Path

# Download video from VIDEO_URL
def download_video(url, out_path="input_video.mp4"):
    ydl_opts = {
        'format': 'bestvideo+bestaudio/best',
        'outtmpl': out_path,
        'quiet': True,
        'merge_output_format': 'mp4',
        # Use the ffmpeg binary already installed
        # This uses the ffmpeg_path variable set earlier
        # If you want to force re-encode to H.264 for compatibility, add postprocessors:
        # 'postprocessors': [{'key': 'FFmpegVideoConvertor', 'preferedformat': 'mp4'}],
        'ffmpeg_location': ffmpeg_path,
        'postprocessors': [{
                'key': 'FFmpegVideoConvertor',
                'preferedformat': 'mp4'
        }],
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])
    return out_path

video_path = download_video("https://www.youtube.com/watch?v=-D5lO5Pl-3Q")

# Load YOLO model
model = YOLO(MODEL_PATH)

# Open video
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    raise RuntimeError(f"Could not open video: {video_path}")

while True:
    ok, frame = cap.read()
    if not ok:
        break
    results = model(frame, imgsz=IMG_SIZE, conf=CONF, iou=IOU, verbose=False)
    annotated = results[0].plot()
    cv2.imshow("YOLOv8 Football Detection", annotated)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

[av1 @ 0x404d5b00] Your platform doesn't suppport hardware accelerated AV1 decoding.
[av1 @ 0x404d5b00] Failed to get pixel format.
[av1 @ 0x404d5b00] Missing Sequence Header.
[av1 @ 0x404d5b00] Missing Sequence Header.
[av1 @ 0x404d5b00] Missing Sequence Header.
[av1 @ 0x404d5b00] Missing Sequence Header.
[av1 @ 0x404d5b00] Missing Sequence Header.
[av1 @ 0x404d5b00] Missing Sequence Header.
[av1 @ 0x404d5b00] Missing Sequence Header.
[av1 @ 0x404d5b00] Missing Sequence Header.
[av1 @ 0x404d5b00] Missing Sequence Header.
[av1 @ 0x404d5b00] Missing Sequence Header.
[av1 @ 0x404d5b00] Missing Sequence Header.
[av1 @ 0x404d5b00] Missing Sequence Header.
[av1 @ 0x404d5b00] Missing Sequence Header.
[av1 @ 0x404d5b00] Missing Sequence Header.
[av1 @ 0x404d5b00] Missing Sequence Header.
[av1 @ 0x404d5b00] Missing Sequence Header.
[av1 @ 0x404d5b00] Missing Sequence Header.
[av1 @ 0x404d5b00] Missing Sequence Header.
[av1 @ 0x404d5b00] Missing Sequence Header.
[av1 @ 0x404d5b00] Missing Seque

In [35]:
import sys, os, stat, subprocess, shutil
!{sys.executable} -m pip -q install yt-dlp imageio-ffmpeg ultralytics opencv-python

# Get a real ffmpeg binary and put it on PATH
import imageio_ffmpeg
ffmpeg_path = imageio_ffmpeg.get_ffmpeg_exe()
try:
    os.chmod(ffmpeg_path, os.stat(ffmpeg_path).st_mode | stat.S_IEXEC)
except Exception:
    pass
os.environ["PATH"] = os.path.dirname(ffmpeg_path) + os.pathsep + os.environ.get("PATH","")

print("FFmpeg:", subprocess.check_output([ffmpeg_path, "-version"], text=True).splitlines()[0])


FFmpeg: ffmpeg version 7.0.2-static https://johnvansickle.com/ffmpeg/  Copyright (c) 2000-2024 the FFmpeg developers


In [36]:
from pathlib import Path
from yt_dlp import YoutubeDL

def download_video_h264(url: str, out_dir="downloads") -> Path:
    """
    Download video preferring avc1(H.264)+m4a(AAC) MP4. Falls back to best if needed.
    """
    out_dir = Path(out_dir); out_dir.mkdir(parents=True, exist_ok=True)
    ydl_opts = {
        "outtmpl": str(out_dir / "%(title).90s.%(ext)s"),
        # Prefer MP4 + H.264 video + m4a audio; then any mp4; else 'best'
        "format": (
            "bestvideo[ext=mp4][vcodec^=avc1]+bestaudio[ext=m4a]/"
            "best[ext=mp4][vcodec^=avc1]/"
            "bestvideo[vcodec^=avc1]+bestaudio/"
            "best"
        ),
        "merge_output_format": "mp4",
        "ffmpeg_location": ffmpeg_path,   # point yt-dlp to the actual ffmpeg binary
        "noplaylist": True,
        "quiet": True,
    }
    with YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(url, download=True)
        fn = Path(ydl.prepare_filename(info))
        mp4 = fn.with_suffix(".mp4")
        return mp4 if mp4.exists() else fn


In [37]:
import subprocess, shlex, cv2

def ensure_readable_mp4(src: Path, force_reencode=False) -> Path:
    """
    Make sure OpenCV can open the file. If not, or if force_reencode=True,
    transcode to H.264 (libx264) + AAC in an MP4 container.
    """
    src = Path(src)

    def can_open(p: Path) -> bool:
        cap = cv2.VideoCapture(str(p), cv2.CAP_FFMPEG)
        ok = cap.isOpened()
        cap.release()
        return ok

    if not force_reencode and can_open(src):
        return src

    # Try fast remux first
    remux = src.with_suffix(".remux.mp4")
    cmd = [ffmpeg_path, "-y", "-i", str(src), "-c", "copy", "-movflags", "+faststart", str(remux)]
    try:
        subprocess.run(cmd, check=True, capture_output=True)
        if not force_reencode and can_open(remux):
            return remux
    except subprocess.CalledProcessError:
        pass

    # Re-encode to H.264/AAC (yuv420p = widely compatible)
    reenc = src.with_suffix(".h264.mp4")
    cmd = [
        ffmpeg_path, "-y", "-i", str(src),
        "-c:v", "libx264", "-pix_fmt", "yuv420p",
        "-c:a", "aac", "-movflags", "+faststart",
        str(reenc)
    ]
    subprocess.run(cmd, check=True)
    if not can_open(reenc):
        raise RuntimeError("Re-encoded MP4 still unreadable by OpenCV.")
    return reenc


In [ ]:
import cv2, time
from ultralytics import YOLO

# ==== EDIT THESE ====
VIDEO_URL   = "https://www.youtube.com/watch?v=-D5lO5Pl-3Q"
MODEL_PATH  = "runs/train/soccer_players_clean-exp/weights/best.pt"
CONF        = 0.20       # try 0.10–0.35 if detections are weak
IOU         = 0.60
IMG_SIZE    = 960        # 960/1280 help with small players/ball
USE_TTA     = False
SHOW_WINDOW = False
OUT_DIR     = Path("runs/video_pred")
# =====================

OUT_DIR.mkdir(parents=True, exist_ok=True)

# 1) Download with avc1 preference
print("Downloading (preferring H.264)…")
local_file = download_video_h264(VIDEO_URL)
print("Saved:", local_file)

# 2) Ensure OpenCV can read (remux/re-encode if needed)
print("Ensuring OpenCV-readable MP4…")
readable = ensure_readable_mp4(local_file)
print("Using:", readable)

# 3) Open and annotate
cap = cv2.VideoCapture(str(readable), cv2.CAP_FFMPEG)
if not cap.isOpened():
    raise RuntimeError("Failed to open video after conversion.")

fps    = cap.get(cv2.CAP_PROP_FPS) or 25.0
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH) or 1280)
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT) or 720)
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out_path = OUT_DIR / f"annotated_{int(time.time())}.mp4"
writer = cv2.VideoWriter(str(out_path), fourcc, fps, (width, height))
if not writer.isOpened():
    raise RuntimeError("VideoWriter failed; check codecs.")

model = YOLO(MODEL_PATH)

t0 = time.time(); frames = 0
while True:
    ok, frame = cap.read()
    if not ok:
        break
    results = model(frame, conf=CONF, iou=IOU, imgsz=IMG_SIZE, augment=USE_TTA, verbose=False)
    annotated = results[0].plot()

    frames += 1
    if frames % 10 == 0:
        fps_cur = frames / (time.time() - t0 + 1e-9)
        cv2.putText(annotated, f"FPS~{fps_cur:.1f} conf:{CONF:.2f} imgsz:{IMG_SIZE} TTA:{USE_TTA}",
                    (10, 28), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0), 2, cv2.LINE_AA)

    writer.write(annotated)
    if SHOW_WINDOW:
        cv2.imshow("YOLOv8 Video", annotated)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
writer.release()
if SHOW_WINDOW:
    cv2.destroyAllWindows()

print("✅ Saved annotated video to:", out_path.resolve())


Saved: downloads/FC Dallas vs. LAFC ｜ Full Match Highlights ｜ Son Heung-Min BANGER!!.mp4
Ensuring OpenCV-readable MP4…
Using: downloads/FC Dallas vs. LAFC ｜ Full Match Highlights ｜ Son Heung-Min BANGER!!.mp4


In [6]:
import cv2
import numpy as np
import requests
from urllib.parse import urlparse
from pathlib import Path
from ultralytics import YOLO

# ===== EDIT =====
MODEL_PATH = "runs/train/soccer_players_clean-exp/weights/best.pt"
IMG_SIZE   = 960
CONF       = 0.25
IOU        = 0.60
# ===============

def _is_url(s: str) -> bool:
    try:
        u = urlparse(s)
        return u.scheme in ("http", "https") and u.netloc != ""
    except Exception:
        return False

def _load_image(url_or_path: str):
    s = str(url_or_path).strip()

    # Common mistake: "/https://..." -> strip leading slashes
    if s.startswith("/http"):
        s = s.lstrip("/")

    if _is_url(s):
        # Download with a browser-like UA; some CDNs reject default Python UA
        headers = {"User-Agent": "Mozilla/5.0"}
        r = requests.get(s, headers=headers, timeout=20)
        r.raise_for_status()
        arr = np.frombuffer(r.content, dtype=np.uint8)
        img = cv2.imdecode(arr, cv2.IMREAD_COLOR)
        if img is None:
            raise RuntimeError("Downloaded bytes could not be decoded as an image.")
        return img, s
    else:
        img = cv2.imread(s)
        if img is None:
            raise FileNotFoundError(f"Could not read image from path: {s}")
        return img, s

def detect_image(url_or_path, show=True, save_path=None):
    model = YOLO(MODEL_PATH)

    img, src = _load_image(url_or_path)
    results = model(img, imgsz=IMG_SIZE, conf=CONF, iou=IOU, verbose=False)
    annotated = results[0].plot()

    # Print detections
    boxes = results[0].boxes
    if boxes is None or len(boxes) == 0:
        print("❌ No detections.")
    else:
        print("✅ Detections:")
        for b in boxes:
            cls = int(b.cls[0])
            conf = float(b.conf[0])
            name = model.names.get(cls, str(cls)) if hasattr(model, "names") else str(cls)
            print(f" - {name}: {conf:.2f}")

    # Decide output path
    if save_path is None:
        # derive a filename from the URL/path (fallback to result.jpg)
        parsed = urlparse(src)
        fname = Path(parsed.path).name or "result.jpg"
        out = Path("runs/detect/single") / fname
        out = out.with_suffix(".jpg")  # make sure it's an image extension
        out.parent.mkdir(parents=True, exist_ok=True)
        save_path = out

    cv2.imwrite(str(save_path), annotated)
    print("💾 Saved annotated image to:", save_path.resolve())

    if show:
        cv2.imshow("YOLOv8 Detection", annotated)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

# -------- Example usage (NOTICE: no leading slash) --------
detect_image("https://tse3.mm.bing.net/th/id/OIP.nvBzlZT0DJGp0ey4-bmUCwAAAA?rs=1&pid=ImgDetMain&o=7&rm=3")


✅ Detections:
 - 0: 0.48
 - 0: 0.47
 - 0: 0.47
💾 Saved annotated image to: /home/surya/c/runs/detect/single/OIP.jpg


qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/surya/c/.venv/lib/python3.12/site-packages/cv2/qt/plugins"


Using image: tmp_url.jpg
Results saved to runs/detect/predict4
[conf=0.05] detections=7  -> saved at: runs/detect/predict4
  box: 0 0.39
  box: 0 0.21
  box: 0 0.17
  box: 0 0.15
  box: 0 0.09
  box: 1 0.07
  box: 1 0.05

Open the newest file inside: runs/detect/predict4
   runs/detect/predict4/tmp_url.jpg


In [11]:
from pathlib import Path
from ultralytics import YOLO

MODEL_PATH = "runs/train/soccer_players_clean-exp/weights/best.pt"
TRAIN_IMG_DIR = Path("/home/surya/Downloads/soccer players.v1i.yolov8/train/images")  # adjust if needed

model = YOLO(MODEL_PATH)
imgs = sorted([p for p in TRAIN_IMG_DIR.glob("*") if p.suffix.lower() in {".jpg",".jpeg",".png",".bmp"}])[:8]
print("Testing on", len(imgs), "training images")

for p in imgs:
    res = model.predict(source=str(p), imgsz=1280, conf=0.10, iou=0.60, save=True, show=False, verbose=False)
    n = 0 if res[0].boxes is None else len(res[0].boxes)
    print(p.name, "->", n, "detections; saved:", res[0].save_dir)


Testing on 8 training images
Results saved to runs/detect/predict5
1-fps-2_00002_jpeg_jpg.rf.05c4a23f214f0186a4f41642c287200c.jpg -> 21 detections; saved: runs/detect/predict5
Results saved to runs/detect/predict5
1-fps-2_00006_jpeg_jpg.rf.e687460a906357f97cfcd1d2fc481219.jpg -> 16 detections; saved: runs/detect/predict5
Results saved to runs/detect/predict5
1-fps-2_00008_jpeg_jpg.rf.14e0ee0c5d38ba5fc78402e000e3b66c.jpg -> 9 detections; saved: runs/detect/predict5
Results saved to runs/detect/predict5
1-fps-2_00009_jpeg_jpg.rf.4e13eea75a7acdd1eb847744fdff39c4.jpg -> 9 detections; saved: runs/detect/predict5
Results saved to runs/detect/predict5
1-fps-2_00010_jpeg_jpg.rf.98f0fb890d12f95d8ca1d47921805b30.jpg -> 9 detections; saved: runs/detect/predict5
Results saved to runs/detect/predict5
1-fps-2_00011_jpeg_jpg.rf.4bbffbe52093c2d7698d8115eee14ae2.jpg -> 10 detections; saved: runs/detect/predict5
Results saved to runs/detect/predict5
1-fps-2_00012_jpeg_jpg.rf.bc0c636c0c9a533aa6974eabb0f4

In [ ]:
import os, requests
from pathlib import Path
from ultralytics import YOLO

MODEL_PATH = "runs/train/soccer_players_clean-exp/weights/best.pt"  # your trained model
IMG_SIZE   = 1280  # bigger helps tiny players/ball
IOU        = 0.60

def fetch_local(src: str) -> str:
    s = src.strip()
    if s.startswith("/http"):  # common typo fix
        s = s.lstrip("/")
    if s.startswith("http://") or s.startswith("https://"):
        r = requests.get(s, headers={"User-Agent": "Mozilla/5.0"}, timeout=20)
        r.raise_for_status()
        fn = Path("tmp_url.jpg")
        fn.write_bytes(r.content)
        return str(fn)
    return s

def test_image(src: str):
    model = YOLO(MODEL_PATH)
    local = fetch_local(src)
    print("Using image:", local)

    # Try a ladder of conf thresholds to avoid “0 detections” hiding boxes
    tried = []
    for conf in [0.05, 0.10, 0.15, 0.20, 0.25]:
        res = model.predict(
            source=local,
            imgsz=IMG_SIZE,
            conf=conf,
            iou=IOU,
            agnostic_nms=True,
            max_det=300,
            save=True,     # <-- saves annotated image/video
            show=False,
            verbose=False
        )
        n = 0 if res[0].boxes is None else len(res[0].boxes)
        out_dir = res[0].save_dir
        tried.append((conf, n, out_dir))
        print(f"[conf={conf:.2f}] detections={n}  -> saved at: {out_dir}")

        # If anything was detected, stop early
        if n > 0:
            # Also print the detections
            for b in res[0].boxes:
                cls = int(b.cls[0]) if b.cls is not None else -1
                confv = float(b.conf[0])
                print("  box:", res[0].names.get(cls, str(cls)), f"{confv:.2f}")
            break

    # Tell user where to look
    last_dir = tried[-1][2]
    print("\nOpen the newest file inside:", last_dir)
    for p in sorted(Path(last_dir).glob("*"))[-5:]:
        print("  ", p)

# EXAMPLE (put your URL here)
test_image("https://tse3.mm.bing.net/th/id/OIP.nvBzlZT0DJGp0ey4-bmUCwAAAA?rs=1&pid=ImgDetMain&o=7&rm=3")


Using image: tmp_url.jpg
Results saved to runs/detect/predict6
[conf=0.05] detections=7  -> saved at: runs/detect/predict6
  box: 0 0.39
  box: 0 0.21
  box: 0 0.17
  box: 0 0.15
  box: 0 0.09
  box: 1 0.07
  box: 1 0.05

Open the newest file inside: runs/detect/predict6
   runs/detect/predict6/tmp_url.jpg


In [14]:
# Real-time detection from a VIDEO URL using your existing YOLO model (version-safe: no 'tta' arg)
# Hotkeys:
#   q  = quit
#   -  = lower confidence
#   =  = raise confidence
#   [  = smaller imgsz
#   ]  = larger imgsz
#   a  = toggle inference-time augmentation (if your Ultralytics build supports 'augment')

import os, sys, time, stat, subprocess
from pathlib import Path
import cv2
from ultralytics import YOLO

# ============ EDIT THESE ============
VIDEO_URL  = "https://www.youtube.com/watch?v=-D5lO5Pl-3Q"   # your link
MODEL_PATH = "runs/train/soccer_players_clean-exp/weights/best.pt"
CONF       = 0.20
IOU        = 0.60
IMG_SIZE   = 960
USE_AUGMENT = False   # toggled with 'a'
SHOW       = True
# ====================================

# --- Ensure a real ffmpeg is available ---
import imageio_ffmpeg
ffmpeg_path = imageio_ffmpeg.get_ffmpeg_exe()
try:
    os.chmod(ffmpeg_path, os.stat(ffmpeg_path).st_mode | stat.S_IEXEC)
except Exception:
    pass
os.environ["PATH"] = os.path.dirname(ffmpeg_path) + os.pathsep + os.environ.get("PATH","")

# --- yt-dlp helpers: direct progressive stream if possible; else download/convert ---
from yt_dlp import YoutubeDL

def get_direct_stream(url: str):
    ydl_opts = {"quiet": True, "noplaylist": True}
    with YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(url, download=False)
        fmts = info.get("formats", [])
        cand = [f for f in fmts
                if f.get("ext")=="mp4"
                and (f.get("vcodec","").startswith("avc1"))
                and (f.get("acodec","none") != "none")
                and f.get("url")]
        if not cand:
            return None
        cand.sort(key=lambda f: (f.get("height") or 0), reverse=True)
        return cand[0]["url"]

def download_video_h264(url: str, out_dir="downloads") -> Path:
    out_dir = Path(out_dir); out_dir.mkdir(parents=True, exist_ok=True)
    ydl_opts = {
        "outtmpl": str(out_dir / "%(title).90s.%(ext)s"),
        "format": (
            "bestvideo[ext=mp4][vcodec^=avc1]+bestaudio[ext=m4a]/"
            "best[ext=mp4][vcodec^=avc1]/"
            "best"
        ),
        "merge_output_format": "mp4",
        "ffmpeg_location": os.path.dirname(ffmpeg_path),
        "noplaylist": True,
        "quiet": True,
    }
    with YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(url, download=True)
        fn = Path(ydl.prepare_filename(info))
        mp4 = fn.with_suffix(".mp4")
        return mp4 if mp4.exists() else fn

def ensure_readable_mp4(src: Path) -> Path:
    def can_open(p: Path) -> bool:
        cap = cv2.VideoCapture(str(p), cv2.CAP_FFMPEG)
        ok = cap.isOpened(); cap.release()
        return ok
    if can_open(src):
        return src
    # remux
    remux = src.with_suffix(".remux.mp4")
    subprocess.run([ffmpeg_path, "-y", "-i", str(src), "-c", "copy",
                    "-movflags", "+faststart", str(remux)],
                   check=False, capture_output=True, text=True)
    if can_open(remux):
        return remux
    # re-encode
    reenc = src.with_suffix(".h264.mp4")
    subprocess.run([ffmpeg_path, "-y", "-i", str(src),
                    "-c:v", "libx264", "-pix_fmt", "yuv420p",
                    "-c:a", "aac", "-movflags", "+faststart", str(reenc)],
                   check=True)
    if not can_open(reenc):
        raise RuntimeError("Re-encoded MP4 still unreadable by OpenCV.")
    return reenc

def open_video_from_url(url: str):
    print("Trying direct progressive stream …")
    direct = None
    try:
        direct = get_direct_stream(url)
    except Exception as e:
        print("Direct stream lookup failed:", e)
    if direct:
        cap = cv2.VideoCapture(direct, cv2.CAP_FFMPEG)
        if cap.isOpened():
            print("✔ Streaming directly.")
            return cap
        cap.release()
    print("⚠ Streaming failed → downloading then playing …")
    local = download_video_h264(url)
    print("Downloaded to:", local)
    playable = ensure_readable_mp4(local)
    print("Using:", playable)
    cap = cv2.VideoCapture(str(playable), cv2.CAP_FFMPEG)
    if not cap.isOpened():
        raise RuntimeError("Failed to open video even after conversion.")
    return cap

# --- Realtime loop with version-safe inference call (no 'tta') ---
model = YOLO(MODEL_PATH)
cap = open_video_from_url(VIDEO_URL)
fps0 = cap.get(cv2.CAP_PROP_FPS) or 25.0
print(f"Opened stream @ ~{fps0:.1f} FPS")

t0 = time.time(); frames = 0
try:
    while True:
        ok, frame = cap.read()
        if not ok:
            break

        # Version-safe inference: try 'augment' if supported, else plain call
        try:
            results = model(frame, conf=CONF, iou=IOU, imgsz=IMG_SIZE,
                            augment=USE_AUGMENT, verbose=False)
        except TypeError:
            results = model(frame, conf=CONF, iou=IOU, imgsz=IMG_SIZE, verbose=False)

        annotated = results[0].plot()

        frames += 1
        if frames % 10 == 0:
            fps_cur = frames / (time.time() - t0 + 1e-9)
            cv2.putText(annotated, f"FPS~{fps_cur:.1f} conf:{CONF:.2f} imgsz:{IMG_SIZE} augment:{USE_AUGMENT}",
                        (10, 28), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2, cv2.LINE_AA)

        if SHOW:
            cv2.imshow("YOLOv8 URL real-time", annotated)

        k = cv2.waitKey(1) & 0xFF
        if k == ord('q'):
            break
        elif k == ord('-'):
            CONF = max(0.05, CONF - 0.05)
        elif k in (ord('='), ord('+')):
            CONF = min(0.95, CONF + 0.05)
        elif k == ord('['):
            IMG_SIZE = max(320, IMG_SIZE - 64)
        elif k == ord(']'):
            IMG_SIZE = min(1536, IMG_SIZE + 64)
        elif k == ord('a'):
            USE_AUGMENT = not USE_AUGMENT
            print("augment =", USE_AUGMENT)

finally:
    cap.release()
    cv2.destroyAllWindows()
    print("✅ Released resources")


Trying direct progressive stream …


✔ Streaming directly.
Opened stream @ ~30.0 FPS
✅ Released resources


In [1]:
# Smooth real-time detection from VIDEO URL (version-safe, faster)
# Hotkeys:
#   q = quit
#   - / = : lower/raise confidence
#   [ / ] : smaller/larger imgsz
#   a     : toggle inference-time augment (off by default; slower)
#   n / m : process every Nth frame (↓ or ↑)  e.g., n=2 ~ process 15 of 30 FPS

import os, time, stat, subprocess
from pathlib import Path
import cv2
from ultralytics import YOLO

# ================== CONFIG ==================
VIDEO_URL   = "https://www.youtube.com/watch?v=-D5lO5Pl-3Q"
MODEL_PATH  = "runs/train/soccer_players_clean-exp/weights/best.pt"
CONF        = 0.20
IOU         = 0.60
IMG_SIZE    = 960          # 640 is faster; 960/1280 sees smaller players
USE_AUGMENT = False        # keep False for speed
PROCESS_EVERY_N = 2        # process every Nth frame; display all (2 or 3 is a good sweet spot)
TARGET_STREAM_HEIGHT = 720 # request 720p progressive for lighter decode
SHOW       = True
# ============================================

# Ensure ffmpeg (bundled) is on PATH
import imageio_ffmpeg
ffmpeg_path = imageio_ffmpeg.get_ffmpeg_exe()
try: os.chmod(ffmpeg_path, os.stat(ffmpeg_path).st_mode | stat.S_IEXEC)
except Exception: pass
os.environ["PATH"] = os.path.dirname(ffmpeg_path) + os.pathsep + os.environ.get("PATH","")

# Direct progressive stream finder (H.264 + audio) at or below TARGET_STREAM_HEIGHT
from yt_dlp import YoutubeDL
def get_direct_stream(url: str, max_h=720):
    ydl_opts = {"quiet": True, "noplaylist": True}
    with YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(url, download=False)
        fmts = info.get("formats", [])
        prog = [f for f in fmts
                if f.get("ext")=="mp4"
                and (f.get("vcodec","").startswith("avc1"))
                and (f.get("acodec","none")!="none")
                and f.get("url")]
        if not prog:
            return None
        # prefer highest ≤ max_h; else lowest above it
        below = [f for f in prog if (f.get("height") or 0) <= max_h]
        cand = sorted(below, key=lambda f: (f.get("height") or 0), reverse=True) or \
               sorted(prog, key=lambda f: (f.get("height") or 0))
        return cand[0]["url"], cand[0].get("height")

def open_video_from_url(url: str, max_h=720):
    print("Trying direct progressive stream …")
    direct = None
    h_sel  = None
    try:
        direct, h_sel = get_direct_stream(url, max_h=max_h)
    except Exception as e:
        print("Direct stream lookup failed:", e)
    if direct:
        cap = cv2.VideoCapture(direct, cv2.CAP_FFMPEG)
        if cap.isOpened():
            print(f"✔ Streaming {h_sel}p progressive MP4.")
            return cap
        cap.release()
    # Fallback: download (still works, just less “live”)
    print("⚠ Streaming failed → downloading then playing …")
    out_dir = Path("downloads"); out_dir.mkdir(parents=True, exist_ok=True)
    ydl_opts = {
        "outtmpl": str(out_dir / "%(title).90s.%(ext)s"),
        "format": (
            "bestvideo[ext=mp4][vcodec^=avc1][height<=" + str(max_h) + "]+bestaudio[ext=m4a]/"
            "best[ext=mp4][vcodec^=avc1][height<=" + str(max_h) + "]/"
            "best"
        ),
        "merge_output_format": "mp4",
        "ffmpeg_location": os.path.dirname(ffmpeg_path),
        "noplaylist": True,
        "quiet": True,
    }
    with YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(url, download=True)
        fn = Path(ydl.prepare_filename(info)).with_suffix(".mp4")
    playable = fn
    cap = cv2.VideoCapture(str(playable), cv2.CAP_FFMPEG)
    if not cap.isOpened():
        # last-resort re-encode
        reenc = playable.with_suffix(".h264.mp4")
        subprocess.run([ffmpeg_path, "-y", "-i", str(playable),
                        "-c:v", "libx264", "-pix_fmt", "yuv420p",
                        "-c:a", "aac", "-movflags", "+faststart",
                        str(reenc)], check=True)
        cap = cv2.VideoCapture(str(reenc), cv2.CAP_FFMPEG)
    if not cap.isOpened():
        raise RuntimeError("Could not open video.")
    print("Playing downloaded MP4.")
    return cap

# Speed tips for OpenCV backend
try:
    cv2.setNumThreads(0)  # avoid oversubscription on some CPUs
except Exception:
    pass

# Load your model once; try GPU + FP16 (falls back safely)
model = YOLO(MODEL_PATH)

device_kw = {}
try:
    import torch
    if torch.cuda.is_available():
        device_kw["device"] = 0
        # Half precision speeds up a lot on recent GPUs
        device_kw["half"] = True
        torch.backends.cudnn.benchmark = True  # faster convs for fixed shapes
        print("Using CUDA + FP16.")
    else:
        print("CUDA not available; running on CPU.")
except Exception:
    print("Torch not found or GPU probe failed; proceeding with defaults.")

cap = open_video_from_url(VIDEO_URL, max_h=TARGET_STREAM_HEIGHT)
fps0 = cap.get(cv2.CAP_PROP_FPS) or 25.0
w0   = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH) or 1280)
h0   = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT) or 720)
print(f"Opened {w0}x{h0} @ ~{fps0:.1f} FPS")

# Processing loop (skim frames for speed)
last_annotated = None
frame_i = 0
t0 = time.time(); shown = 0

try:
    while True:
        ok, frame = cap.read()
        if not ok:
            break

        # Always display something, but only run inference every Nth frame
        run_infer = (frame_i % max(1, int(PROCESS_EVERY_N)) == 0)
        if run_infer or last_annotated is None:
            # Version-safe: try augment; otherwise plain call
            try:
                results = model(frame, conf=CONF, iou=IOU, imgsz=IMG_SIZE,
                                augment=USE_AUGMENT, agnostic_nms=True,
                                max_det=150, verbose=False, **device_kw)
            except TypeError:
                results = model(frame, conf=CONF, iou=IOU, imgsz=IMG_SIZE,
                                agnostic_nms=True, max_det=150,
                                verbose=False, **device_kw)
            last_annotated = results[0].plot()

        # FPS overlay (display FPS, not inference FPS)
        shown += 1
        if shown % 10 == 0:
            fps_disp = shown / (time.time() - t0 + 1e-9)
            cv2.putText(last_annotated, f"Display~{fps_disp:.1f}  conf:{CONF:.2f}  imgsz:{IMG_SIZE}  N:{PROCESS_EVERY_N}  aug:{USE_AUGMENT}",
                        (10, 28), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2, cv2.LINE_AA)

        if SHOW:
            cv2.imshow("YOLOv8 URL realtime (q quit, -/= conf, [/] size, n/m stride, a aug)", last_annotated)

        k = cv2.waitKey(1) & 0xFF
        if k == ord('q'): break
        elif k == ord('-'): CONF = max(0.01, CONF - 0.05)
        elif k in (ord('='), ord('+')): CONF = min(0.99, CONF + 0.05)
        elif k == ord('['): IMG_SIZE = max(320, IMG_SIZE - 64)
        elif k == ord(']'): IMG_SIZE = min(1536, IMG_SIZE + 64)
        elif k == ord('a'): USE_AUGMENT = not USE_AUGMENT; print("augment =", USE_AUGMENT)
        elif k == ord('n'): PROCESS_EVERY_N = max(1, PROCESS_EVERY_N + 1); print("process every N frames:", PROCESS_EVERY_N)
        elif k == ord('m'): PROCESS_EVERY_N = max(1, PROCESS_EVERY_N - 1); print("process every N frames:", PROCESS_EVERY_N)

        frame_i += 1

finally:
    cap.release()
    cv2.destroyAllWindows()
    print("✅ Released resources")


CUDA not available; running on CPU.
Trying direct progressive stream …


✔ Streaming 360p progressive MP4.
Opened 640x360 @ ~30.0 FPS


qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/surya/c/.venv/lib/python3.12/site-packages/cv2/qt/plugins"


✅ Released resources


In [2]:
# Robust WEBCAM realtime YOLO (multi-backend open + MJPG + warmup + threaded)
# Hotkeys:
#   q : quit
#   - / = : lower/raise confidence
#   [ / ] : smaller/larger inference imgsz
#   n / m : increase/decrease inference stride
#   a     : toggle augment
#   r     : start/stop MP4 recording to runs/webcam/

import os, time, threading, queue
from pathlib import Path
import cv2
from ultralytics import YOLO

# ==================== CONFIG ====================
MODEL_PATH        = "runs/train/soccer_players_clean-exp/weights/best.pt"
PREFERRED_INDEXES = [0]   # try these camera indices in order
TARGET_RES        = (1280, 720)    # request this capture size
TARGET_FPS        = 30
FORCE_MJPG        = True           # MJPG often boosts FPS on 720p/1080p
WARMUP_FRAMES     = 20             # frames to grab before starting (verifies camera is alive)

CONF              = 0.20
IOU               = 0.60
IMG_SIZE          = 960
PROCESS_EVERY_N   = 3
USE_AUGMENT       = False
SHOW              = True
# =================================================

# Optional: avoid CPU oversubscription
try: cv2.setNumThreads(0)
except Exception: pass

# Load model & device
model = YOLO(MODEL_PATH)
device_arg = 0
try:
    import torch
    if not torch.cuda.is_available():
        device_arg = 'cpu'
        print("CUDA not available; running on CPU.")
    else:
        print("Using CUDA for inference.")
        torch.backends.cudnn.benchmark = True
except Exception:
    device_arg = 'cpu'
    print("Torch probe failed; running on CPU.")

# --- helpers ---
def try_open_camera(index, backend):
    cap = cv2.VideoCapture(index, backend)
    if not cap.isOpened():
        return None
    # Request format for speed
    if FORCE_MJPG:
        cap.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc(*'MJPG'))
    cap.set(cv2.CAP_PROP_FRAME_WIDTH,  TARGET_RES[0])
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, TARGET_RES[1])
    cap.set(cv2.CAP_PROP_FPS, TARGET_FPS)
    try: cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)
    except Exception: pass
    return cap

def open_any_camera(indexes):
    # On Linux: prefer V4L2, then ANY
    backends = []
    if os.name != "nt":
        backends = [cv2.CAP_V4L2, cv2.CAP_ANY]
    else:
        backends = [cv2.CAP_DSHOW, cv2.CAP_MSMF, cv2.CAP_ANY]

    for idx in indexes:
        for be in backends:
            cap = try_open_camera(idx, be)
            if cap is None:
                continue
            # Warmup: try to read WARMUP_FRAMES inside 2 seconds
            ok_count = 0
            t0 = time.time()
            for _ in range(WARMUP_FRAMES):
                ok, _ = cap.read()
                if ok: ok_count += 1
                if time.time() - t0 > 2.0:
                    break
            if ok_count >= max(5, WARMUP_FRAMES // 3):
                w  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH) or TARGET_RES[0])
                h  = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT) or TARGET_RES[1])
                fps = cap.get(cv2.CAP_PROP_FPS) or TARGET_FPS
                print(f"✔ Opened camera index {idx} via backend {be} at ~{w}x{h} @ {fps:.1f} fps (warmup ok={ok_count})")
                return cap, idx, be
            else:
                print(f"⚠ Camera {idx} backend {be} produced no/slow frames in warmup (ok={ok_count}); trying next…")
                cap.release()
    return None, None, None

cap, cam_idx, backend = open_any_camera(PREFERRED_INDEXES)
if cap is None:
    raise RuntimeError("❌ Could not open any webcam. Make sure no other app is using it and try a different index.")

# ---------- Threaded reader: keep only latest frame ----------
frame_q: "queue.Queue[tuple[float, any]]" = queue.Queue(maxsize=1)
stop_flag = False

def reader_loop():
    global stop_flag
    while not stop_flag:
        ok, frame = cap.read()
        if not ok:
            time.sleep(0.005)
            continue
        ts = time.time()
        # drop stale frame (maxsize=1)
        if frame_q.full():
            try: frame_q.get_nowait()
            except Exception: pass
        frame_q.put((ts, frame))
    stop_flag = True

t_reader = threading.Thread(target=reader_loop, daemon=True)
t_reader.start()

# Optional MP4 recorder
save_root = Path("runs/webcam"); save_root.mkdir(parents=True, exist_ok=True)
writer, recording = None, False
def start_writer(frame):
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    outp = save_root / f"webcam_{int(time.time())}.mp4"
    h, w = frame.shape[:2]
    fps = cap.get(cv2.CAP_PROP_FPS) or 30.0
    vw = cv2.VideoWriter(str(outp), fourcc, fps, (w, h))
    return vw, outp

# Main loop
last_annot = None
shown = 0
t0 = time.time()
frame_idx = 0

try:
    while True:
        try:
            ts, frame = frame_q.get(timeout=2.0)
        except queue.Empty:
            print("⚠ No frames from camera; still waiting… (is it in use by another app?)")
            continue

        run_infer = (frame_idx % max(1, int(PROCESS_EVERY_N)) == 0) or (last_annot is None)

        if run_infer:
            # Version-safe inference
            try:
                results = model(frame,
                                conf=CONF, iou=IOU, imgsz=IMG_SIZE,
                                augment=USE_AUGMENT, agnostic_nms=True, max_det=120,
                                device=device_arg, verbose=False)
            except TypeError:
                results = model(frame,
                                conf=CONF, iou=IOU, imgsz=IMG_SIZE,
                                agnostic_nms=True, max_det=120,
                                device=device_arg, verbose=False)
            last_annot = results[0].plot()

        # HUD (display FPS)
        shown += 1
        if shown % 10 == 0:
            fps_disp = shown / (time.time() - t0 + 1e-9)
            hud = f"Disp~{fps_disp:.1f}  conf:{CONF:.2f}  imgsz:{IMG_SIZE}  N:{PROCESS_EVERY_N}  aug:{USE_AUGMENT}"
            cv2.putText(last_annot, hud, (10, 28), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2, cv2.LINE_AA)

        if SHOW:
            # Create window once and make sure it stays on top (optional)
            # cv2.namedWindow("YOLOv8 Webcam", cv2.WINDOW_NORMAL)
            cv2.imshow("YOLOv8 Webcam (q quit, -/= conf, [/] size, n/m stride, a aug, r rec)", last_annot)

        # Recording
        if recording:
            if writer is None:
                writer, outfile = start_writer(last_annot)
                print("⏺ Recording to:", outfile)
            writer.write(last_annot)
        elif writer is not None:
            writer.release(); writer = None
            print("⏹ Recording stopped.")

        # Hotkeys
        k = cv2.waitKey(1) & 0xFF
        if k == ord('q'):
            break
        elif k == ord('-'):
            CONF = max(0.01, CONF - 0.05)
        elif k in (ord('='), ord('+')):
            CONF = min(0.99, CONF + 0.05)
        elif k == ord('['):
            IMG_SIZE = max(320, IMG_SIZE - 64)
        elif k == ord(']'):
            IMG_SIZE = min(1536, IMG_SIZE + 64)
        elif k == ord('a'):
            USE_AUGMENT = not USE_AUGMENT; print("augment =", USE_AUGMENT)
        elif k == ord('n'):
            PROCESS_EVERY_N = max(1, PROCESS_EVERY_N + 1); print("process every N frames:", PROCESS_EVERY_N)
        elif k == ord('m'):
            PROCESS_EVERY_N = max(1, PROCESS_EVERY_N - 1); print("process every N frames:", PROCESS_EVERY_N)
        elif k == ord('r'):
            recording = not recording

        frame_idx += 1

finally:
    # Cleanup
    try:
        stop_flag = True
        t_reader.join(timeout=1.0)
    except Exception:
        pass
    if writer is not None:
        writer.release()
    cap.release()
    cv2.destroyAllWindows()
    print("Webcam released. Goodbye!")


CUDA not available; running on CPU.
✔ Opened camera index 0 via backend 200 at ~1280x720 @ 30.0 fps (warmup ok=20)
Webcam released. Goodbye!


In [6]:
%pip -q install --upgrade pip setuptools wheel

# Avoid OpenCV/NumPy conflicts (keep GUI build of OpenCV + NumPy 1.26.x)
%pip -q uninstall -y opencv-python-headless opencv-contrib-python-headless opencv-contrib-python || true
%pip -q install "opencv-python>=4.9,<4.13" "numpy==1.26.4"

# Core libs with Py3.12 wheels
%pip -q install "ultralytics==8.2.0" \
                "onnx>=1.16.1,<1.18" \
                "onnxruntime>=1.17.0,<1.19" \
                "pyyaml" "tqdm" "matplotlib" "pandas"

# Try ONNX simplifier (optional). If no wheel for your platform, we'll skip simplification later.
try:
    import onnxsim  # noqa
    print("onnxsim already installed")
except Exception:
    print("Trying to install onnxsim (optional)…")
    rc = !pip install -q --only-binary=:all: onnxsim
    try:
        import onnxsim  # noqa
        print("onnxsim installed")
    except Exception:
        print("onnxsim not available — simplification will be skipped.")


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Trying to install onnxsim (optional)…
onnxsim not available — simplification will be skipped.


In [1]:
import sys, os
import cv2

# Diagnose what got imported
print("cv2 module:", cv2)
print("cv2.__file__:", getattr(cv2, "__file__", "None"))
print("Has setNumThreads?", hasattr(cv2, "setNumThreads"))

# Common local collisions (rename/delete if found, then restart kernel)
collisions = []
for name in ("cv2.py", "cv2", "opencv.py", "opencv"):
    p = os.path.join(os.getcwd(), name)
    if os.path.exists(p):
        collisions.append(p)
print("Local name collisions:", collisions or "none")

# ---- Compatibility shim: add a no-op setNumThreads if missing ----
if not hasattr(cv2, "setNumThreads"):
    def _noop(*args, **kwargs):  # harmless no-op
        return None
    cv2.setNumThreads = _noop  # type: ignore
    print("Shimmed cv2.setNumThreads → no-op")
else:
    print("cv2.setNumThreads is available")

# (Optional) Also ensure getNumThreads exists to avoid any other surprises
if not hasattr(cv2, "getNumThreads"):
    cv2.getNumThreads = lambda: 0  # type: ignore


cv2 module: <module 'cv2' from '/home/surya/c/.venv/lib/python3.12/site-packages/cv2/__init__.py'>
cv2.__file__: /home/surya/c/.venv/lib/python3.12/site-packages/cv2/__init__.py
Has setNumThreads? True
Local name collisions: none
cv2.setNumThreads is available


In [2]:
import os, time, shutil
from pathlib import Path
import numpy as np
import onnx, onnxruntime as ort
from ultralytics import YOLO

print("onnx:", onnx.__version__, "| onnxruntime:", ort.__version__, "| providers:", ort.get_available_providers())


onnx: 1.17.0 | onnxruntime: 1.18.1 | providers: ['AzureExecutionProvider', 'CPUExecutionProvider']


In [3]:
# Your trained weights
MODEL_PT   = Path("runs/train/soccer_players_clean-exp/weights/best.pt")

# Export settings (Hailo-friendly)
IMG_SIZE   = 960         # choose 640 / 960 / 1280; keep consistent later
OPSET      = 12          # 11–13 fine; 12 is safe
INCLUDE_NMS= False       # Hailo expects raw heads
DYNAMIC    = False       # static [1,3,IMG,IMG] preferred by compilers

# Output dir
EXPORTS_DIR = Path("hailo_build/exports")
EXPORTS_DIR.mkdir(parents=True, exist_ok=True)

assert MODEL_PT.exists(), f"Missing weights: {MODEL_PT}"
print("Using weights:", MODEL_PT)
print("Export dir:", EXPORTS_DIR.resolve())


Using weights: runs/train/soccer_players_clean-exp/weights/best.pt
Export dir: /home/surya/c/hailo_build/exports


In [ ]:
%pip install "torch<2.6" --extra-index-url https://download.pytorch.org/whl/cpu


Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cpu
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.6/174.6 MB 33.8 MB/s  0:00:05:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 38.1 MB/s  0:00:00
  Attempting uninstall: sympy
    Found existing installation: sympy 1.14.0
    Uninstalling sympy-1.14.0:
      Successfully uninstalled sympy-1.14.0━━━━━ 0/2 [sympy]
  Attempting uninstall: torch━━━━━━━━━━━━━━━━━━━ 0/2 [sympy]
    Found existing installation: torch 2.8.0 0/2 [sympy]
    Uninstalling torch-2.8.0:╺━━━━━━━━━━━━━━━━━━━ 1/2 [torch]
      Successfully uninstalled torch-2.8.0m━━━━━━━━━━━━━━━━━━━ 1/2 [torch]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [torch]32m1/2 [torch]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
onnxslim 0.1.65 requires sympy>=1.13.3, but you have sympy 1.13.1 which is incompat

In [ ]:
# --- Patch cv2 & torch.load for this session ---

# 1) Some OpenCV wheels lack setNumThreads; Ultralytics calls it on import.
import cv2
if not hasattr(cv2, "setNumThreads"):
    cv2.setNumThreads = lambda *a, **k: None  # harmless no-op
    print("Shimmed cv2.setNumThreads → no-op")

# 2) PyTorch 2.6 defaults weights_only=True; force False so Ultralytics checkpoints load.
import torch
_orig_torch_load = torch.load
def _torch_load_patched(*args, **kwargs):
    if "weights_only" not in kwargs:
        kwargs["weights_only"] = False  # allow class pickles (you trust this file)
    return _orig_torch_load(*args, **kwargs)
torch.load = _torch_load_patched
print("Patched torch.load ⇒ default weights_only=False")

# 3) (Optional) also allowlist common classes to reduce future prompts
try:
    from torch.serialization import add_safe_globals
    from ultralytics.nn.tasks import DetectionModel, SegmentationModel, ClassificationModel
    from torch.nn.modules.container import Sequential, ModuleList
    add_safe_globals([DetectionModel, SegmentationModel, ClassificationModel, Sequential, ModuleList])
    print("Added safe globals for Ultralytics + torch.nn containers")
except Exception as e:
    print("Safe-globals add skipped:", e)


Patched torch.load ⇒ default weights_only=False
Added safe globals for Ultralytics + torch.nn containers


In [13]:
# === Robust ONNX export (fixes Torch 2.6 unpickler + missing DFLoss + cv2 threads) ===
import os, time, shutil, importlib
from pathlib import Path

# ---- EDIT if your run path differs ----
MODEL_PT    = Path("runs/train/soccer_players_clean-exp2/weights/best.pt")
IMG_SIZE    = 960
OPSET       = 12
INCLUDE_NMS = False  # Hailo expects raw heads
DYNAMIC     = False  # static [1,3,IMG,IMG] for hardware compilers
EXPORTS_DIR = Path("hailo_build/exports"); EXPORTS_DIR.mkdir(parents=True, exist_ok=True)
# ---------------------------------------

# 0) OpenCV shim: Ultralytics calls cv2.setNumThreads on import
import cv2
if not hasattr(cv2, "setNumThreads"):
    cv2.setNumThreads = lambda *a, **k: None  # harmless no-op
    print("Shimmed cv2.setNumThreads → no-op")

# 1) PyTorch 2.6: force weights_only=False for this session (you trust your file)
import torch
if not hasattr(torch, "_orig_load"):
    torch._orig_load = torch.load
def _torch_load_patched(*args, **kwargs):
    if "weights_only" not in kwargs:
        kwargs["weights_only"] = False
    return torch._orig_load(*args, **kwargs)
torch.load = _torch_load_patched
print("Patched torch.load ⇒ default weights_only=False")

# 2) Pre-import Ultralytics & inject DFLoss stub if missing
#    (not used at inference/export; just satisfies unpickler for older checkpoints)
try:
    from ultralytics.utils import loss as uloss
    import torch.nn as nn
    if not hasattr(uloss, "DFLoss"):
        class DFLoss(nn.Module):
            def __init__(self, *a, **k): super().__init__()
            def forward(self, *a, **k):  # should never be called during export/inference
                raise RuntimeError("DFLoss stub was called unexpectedly.")
        uloss.DFLoss = DFLoss
        print("Injected stub ultralytics.utils.loss.DFLoss")
except Exception as e:
    print("Note: could not import ultralytics.utils.loss yet:", e)

# 3) Allowlist common classes to reduce further prompts (optional)
try:
    from torch.serialization import add_safe_globals
    from torch.nn.modules.container import Sequential, ModuleList
    from ultralytics.nn.tasks import DetectionModel, SegmentationModel, ClassificationModel
    allow = [DetectionModel, SegmentationModel, ClassificationModel, Sequential, ModuleList]
    try:
        from ultralytics.nn.tasks import PoseModel
        allow.append(PoseModel)
    except Exception:
        pass
    add_safe_globals(allow)
    print("Added safe globals for Ultralytics + torch.nn containers")
except Exception as e:
    print("Safe-globals add skipped:", e)

# 4) Load YOLO and export ONNX
from ultralytics import YOLO
assert MODEL_PT.exists(), f"Missing weights: {MODEL_PT}"
print("Loading weights:", MODEL_PT)
model = YOLO(str(MODEL_PT))

print("Exporting ONNX…")
onnx_path = model.export(
    format   = "onnx",
    imgsz    = IMG_SIZE,
    opset    = OPSET,
    nms      = INCLUDE_NMS,   # keep False for Hailo
    dynamic  = DYNAMIC,       # keep static shape
    simplify = False          # simplifier optional; skip for compatibility
)

# 5) Persist into our exports dir
onnx_path = Path(onnx_path)
onnx_out  = EXPORTS_DIR / f"{onnx_path.stem}.onnx"
if onnx_path.resolve() != onnx_out.resolve():
    shutil.copy2(onnx_path, onnx_out)
print("✅ ONNX exported to:", onnx_out)

# 6) Verify structure & print I/O names (needed for Hailo YAML)
import onnx
m = onnx.load(str(onnx_out))
onnx.checker.check_model(m)
def shp(v):  # pretty shape
    return [d.dim_value if d.HasField("dim_value") else "?" for d in v.type.tensor_type.shape.dim]
inputs  = list(m.graph.input)
outputs = list(m.graph.output)
print("\nInputs:")
for i in inputs:  print(" -", i.name, shp(i))
print("Outputs:")
for o in outputs: print(" -", o.name, shp(o))
ONNX_INPUT_NAME = inputs[0].name
ONNX_OUTPUTS    = [o.name for o in outputs]
print("\nSummary → input:", ONNX_INPUT_NAME, "| outputs:", ONNX_OUTPUTS)
print("\nAll done. You can now proceed with Hailo Model Compiler.")

Patched torch.load ⇒ default weights_only=False
Added safe globals for Ultralytics + torch.nn containers
Loading weights: runs/train/soccer_players_clean-exp2/weights/best.pt
Exporting ONNX…
Ultralytics YOLOv8.2.0 🚀 Python-3.12.3 torch-2.5.1+cpu CPU (11th Gen Intel Core(TM) i5-11300H 3.10GHz)
Model summary (fused): 168 layers, 3006233 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from 'runs/train/soccer_players_clean-exp2/weights/best.pt' with input shape (1, 3, 960, 960) BCHW and output shape(s) (1, 7, 18900) (5.9 MB)

ONNX: starting export with onnx 1.17.0 opset 12...
ONNX: export success ✅ 0.8s, saved as 'runs/train/soccer_players_clean-exp2/weights/best.onnx' (11.9 MB)

Export complete (2.9s)
Results saved to /home/surya/c/runs/train/soccer_players_clean-exp2/weights
Predict:         yolo predict task=detect model=runs/train/soccer_players_clean-exp2/weights/best.onnx imgsz=960  
Validate:        yolo val task=detect model=runs/train/soccer_players_clean-exp2/weights/best.

In [14]:
from pathlib import Path
import os, yaml, time, shutil, random
import onnx
from ultralytics import YOLO

# ==== EDIT THESE IF NEEDED ====
EXPORTS_DIR  = Path("hailo_build/exports")
ONNX_CANDID  = sorted(EXPORTS_DIR.glob("*.onnx"), key=lambda p: p.stat().st_mtime, reverse=True)
assert ONNX_CANDID, "No ONNX found. Run the ONNX export cells first."
ONNX_PATH    = ONNX_CANDID[0]   # latest

DATASET_ROOT = Path("/home/surya/Downloads/soccer players.v1i.yolov8")  # your dataset folder
RUN_DIR      = Path("runs/train/soccer_players_clean-exp")
MODEL_PT     = RUN_DIR / "weights/best.pt"
IMG_SIZE     = 960              # must match your ONNX export size
CALIB_MAX    = 800              # number of calibration images to collect

WORK_DIR     = Path("hailo_build")
CALIB_DIR    = WORK_DIR / "calib"
DFC_YAML     = WORK_DIR / "model.yaml"
HEF_DIR      = WORK_DIR / "hef"
for p in (CALIB_DIR, HEF_DIR): p.mkdir(parents=True, exist_ok=True)

print("Using ONNX:", ONNX_PATH)

# Parse ONNX I/O names (needed for DFC YAML)
m = onnx.load(str(ONNX_PATH))
inputs  = list(m.graph.input)
outputs = list(m.graph.output)
def shp(v): return [d.dim_value if d.HasField("dim_value") else "?" for d in v.type.tensor_type.shape.dim]
print("Inputs:")
for i in inputs:  print(" -", i.name, shp(i))
print("Outputs:")
for o in outputs: print(" -", o.name, shp(o))

ONNX_INPUT_NAME = inputs[0].name
ONNX_OUTPUTS    = [o.name for o in outputs]

# Find class names
def find_data_yaml(root: Path):
    if (root / "data.yaml").exists(): return root / "data.yaml"
    for q in root.rglob("data.yaml"):
        return q
    if (RUN_DIR / "data.yaml").exists(): return RUN_DIR / "data.yaml"
    for q in RUN_DIR.rglob("data.yaml"):
        return q
    return None

CLASS_NAMES, NUM_CLASSES = [], 80
dy = find_data_yaml(DATASET_ROOT)
if dy and dy.exists():
    spec = yaml.safe_load(dy.read_text())
    nm = spec.get("names", [])
    if isinstance(nm, dict): CLASS_NAMES = [nm[i] for i in sorted(map(int, nm.keys()))]
    elif isinstance(nm, list): CLASS_NAMES = nm
    if CLASS_NAMES: NUM_CLASSES = len(CLASS_NAMES)
elif MODEL_PT.exists():
    y = YOLO(str(MODEL_PT))
    names = y.names if isinstance(y.names, dict) else {i:n for i,n in enumerate(y.names)}
    CLASS_NAMES = [names[i] for i in sorted(names.keys())]
    NUM_CLASSES = len(CLASS_NAMES)

print("Classes:", CLASS_NAMES if CLASS_NAMES else "(unknown)")
print("num_classes:", NUM_CLASSES)


Using ONNX: hailo_build/exports/best.onnx
Inputs:
 - images [1, 3, 960, 960]
Outputs:
 - output0 [1, 7, 18900]
Classes: ['0', '1', '2']
num_classes: 3


In [15]:
import shutil, random
from tqdm import tqdm

CALIB_EXTS = {".jpg",".jpeg",".png",".bmp",".tif",".tiff"}
CALIB_DIR.mkdir(parents=True, exist_ok=True)

candidates = []
for p in DATASET_ROOT.rglob("*"):
    if p.is_file() and p.suffix.lower() in CALIB_EXTS:
        candidates.append(p)

random.seed(42)
random.shuffle(candidates)
pick = candidates[:CALIB_MAX] if len(candidates) > CALIB_MAX else candidates

copied = 0
for src in tqdm(pick, desc="Copying calibration images"):
    try:
        dst = CALIB_DIR / src.name
        if not dst.exists():
            shutil.copy2(src, dst)
        copied += 1
    except Exception:
        pass

print(f"✅ Calibration images prepared: {copied} at {CALIB_DIR.resolve()}")


Copying calibration images: 100%|██████████| 163/163 [00:00<00:00, 2179.03it/s]

✅ Calibration images prepared: 163 at /home/surya/c/hailo_build/calib


In [16]:
import numpy as np, cv2, time
from pathlib import Path

# Keep consistent with export
IMG_SIZE = 960
CLASS_NAMES = globals().get("CLASS_NAMES", [])  # from earlier; else set manually: ["player","ball","referee",...]
CONF_THR = 0.25
IOU_THR  = 0.60
MAX_DET  = 300

def letterbox(im, new_size=640, color=(114,114,114), stride=32):
    """Resize + pad to square while preserving aspect ratio; returns image, ratio, padding."""
    shape = im.shape[:2]  # (h,w)
    if isinstance(new_size, int):
        new_size = (new_size, new_size)
    r = min(new_size[0] / shape[0], new_size[1] / shape[1])
    new_unpad = (int(round(shape[1] * r)), int(round(shape[0] * r)))
    dw, dh = new_size[1] - new_unpad[0], new_size[0] - new_unpad[1]
    dw /= 2; dh /= 2
    if shape[::-1] != new_unpad:
        im = cv2.resize(im, new_unpad, interpolation=cv2.INTER_LINEAR)
    top, bottom = int(round(dh-0.1)), int(round(dh+0.1))
    left, right = int(round(dw-0.1)), int(round(dw+0.1))
    im = cv2.copyMakeBorder(im, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)
    return im, r, (left, top)

def nms_numpy(boxes, scores, iou_thr=0.5, max_det=300):
    """Classic NMS in NumPy on xyxy boxes."""
    if boxes.size == 0:
        return []
    x1, y1, x2, y2 = boxes.T
    areas = (x2 - x1) * (y2 - y1)
    order = scores.argsort()[::-1]
    keep = []
    while order.size > 0 and len(keep) < max_det:
        i = order[0]
        keep.append(i)
        xx1 = np.maximum(x1[i], x1[order[1:]])
        yy1 = np.maximum(y1[i], y1[order[1:]])
        xx2 = np.minimum(x2[i], x2[order[1:]])
        yy2 = np.minimum(y2[i], y2[order[1:]])
        w = np.maximum(0.0, xx2 - xx1)
        h = np.maximum(0.0, yy2 - yy1)
        inter = w * h
        iou = inter / (areas[i] + areas[order[1:]] - inter + 1e-9)
        inds = np.where(iou <= iou_thr)[0]
        order = order[inds + 1]
    return keep

def draw_dets(frame, dets):
    """dets: list of (xyxy, conf, cls_id)."""
    for (x1,y1,x2,y2), conf, cid in dets:
        name = CLASS_NAMES[cid] if CLASS_NAMES and cid < len(CLASS_NAMES) else str(cid)
        cv2.rectangle(frame, (int(x1),int(y1)), (int(x2),int(y2)), (0,255,0), 2)
        cv2.putText(frame, f"{name} {conf:.2f}", (int(x1), max(0,int(y1)-5)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2, cv2.LINE_AA)
    return frame


In [17]:
import onnxruntime as ort

# Prefer CUDA if available
providers = ["CPUExecutionProvider"]
try:
    if "CUDAExecutionProvider" in ort.get_available_providers():
        providers = ["CUDAExecutionProvider", "CPUExecutionProvider"]
        print("Using ORT CUDAExecutionProvider")
except Exception:
    pass

# Find the latest ONNX exported earlier
EXPORTS_DIR = Path("hailo_build/exports")
onnx_list = sorted(EXPORTS_DIR.glob("*.onnx"), key=lambda p: p.stat().st_mtime, reverse=True)
assert onnx_list, "No ONNX in hailo_build/exports"
ONNX_PATH = onnx_list[0]
print("Using ONNX:", ONNX_PATH)

sess = ort.InferenceSession(str(ONNX_PATH), providers=providers)
inp = sess.get_inputs()[0]
inp_name = inp.name
print("Input:", inp_name, "shape:", [d if isinstance(d,int) else '?' for d in inp.shape])

def preprocess(frame_bgr):
    img, r, (padw, padh) = letterbox(frame_bgr, new_size=IMG_SIZE)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    x = img_rgb.astype(np.float32) / 255.0
    x = np.transpose(x, (2,0,1))[None, ...]  # [1,3,H,W]
    return x, r, padw, padh, img.shape[1], img.shape[0]

def postprocess(ort_outs, r, padw, padh, orig_w, orig_h, conf_thr=CONF_THR, iou_thr=IOU_THR):
    """
    Assumes Ultralytics-style output: [1, N, 5+C] with columns [cx,cy,w,h, obj, class_probs...]
    If your export produced a list, we use the first with rank 3.
    """
    y = None
    if isinstance(ort_outs, (list,tuple)):
        for t in ort_outs:
            if isinstance(t, np.ndarray) and t.ndim == 3 and t.shape[0] == 1:
                y = t
                break
    elif isinstance(ort_outs, np.ndarray):
        y = ort_outs
    assert y is not None, f"Unexpected ONNX outputs: {[o.shape if hasattr(o,'shape') else type(o) for o in (ort_outs if isinstance(ort_outs,(list,tuple)) else [ort_outs])]}"

    preds = y[0]  # [N, 5+C]
    if preds.shape[1] < 6:
        return []

    b = preds[:, :4]             # cx,cy,w,h (on padded/letterboxed canvas IMG_SIZE)
    obj = preds[:, 4:5]          # [N,1]
    cls = preds[:, 5:]           # [N,C]
    scores = obj * cls           # [N,C]
    cls_ids = scores.argmax(axis=1)
    confs = scores.max(axis=1)

    # threshold
    m = confs >= conf_thr
    if not np.any(m):
        return []
    b = b[m]; confs = confs[m]; cls_ids = cls_ids[m]

    # xywh → xyxy on letterboxed canvas
    cx, cy, w, h = b[:,0], b[:,1], b[:,2], b[:,3]
    x1 = cx - w/2; y1 = cy - h/2; x2 = cx + w/2; y2 = cy + h/2
    boxes = np.stack([x1, y1, x2, y2], axis=1)

    # map back from letterbox canvas (IMG_SIZE) to original frame
    # undo padding, then divide by r
    boxes[:, [0,2]] -= padw
    boxes[:, [1,3]] -= padh
    boxes /= r

    # clip
    boxes[:, [0,2]] = boxes[:, [0,2]].clip(0, orig_w-1)
    boxes[:, [1,3]] = boxes[:, [1,3]].clip(0, orig_h-1)

    # NMS per-class (simple way: single-class NMS on scores; or do per-class loop)
    keep = nms_numpy(boxes, confs, iou_thr=iou_thr, max_det=MAX_DET)
    boxes = boxes[keep]; confs = confs[keep]; cls_ids = cls_ids[keep]

    return [ (boxes[i], float(confs[i]), int(cls_ids[i])) for i in range(len(keep)) ]


Using ONNX: hailo_build/exports/best.onnx
Input: images shape: [1, 3, 960, 960]


In [20]:
# Open webcam
SRC_INDEX = 0  # change if you have multiple cams
cap = cv2.VideoCapture(SRC_INDEX)
assert cap.isOpened(), "Could not open webcam"

# ask for 1280x720; driver may adjust
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
cap.set(cv2.CAP_PROP_FPS, 30)

shown, t0 = 0, time.time()
try:
    while True:
        ok, frame = cap.read()
        if not ok:
            break
        x, r, padw, padh, Wc, Hc = preprocess(frame)
        outs = sess.run(None, {inp_name: x})
        dets = postprocess(outs, r, padw, padh, frame.shape[1], frame.shape[0], CONF_THR, IOU_THR)
        out = draw_dets(frame.copy(), dets)

        shown += 1
        if shown % 10 == 0:
            fps = shown / (time.time() - t0 + 1e-9)
            cv2.putText(out, f"FPS~{fps:.1f} conf:{CONF_THR:.2f} iou:{IOU_THR:.2f} imgsz:{IMG_SIZE}",
                        (10,28), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2, cv2.LINE_AA)

        cv2.imshow("ONNX YOLOv8 — Webcam (q quit, -/= conf, [/] size)", out)
        k = cv2.waitKey(1) & 0xFF
        if k == ord('q'): break
        elif k == ord('-'): CONF_THR = max(0.01, CONF_THR - 0.05)
        elif k in (ord('='), ord('+')): CONF_THR = min(0.99, CONF_THR + 0.05)
        elif k == ord('['): IMG_SIZE = max(320, IMG_SIZE - 64)
        elif k == ord(']'): IMG_SIZE = min(1536, IMG_SIZE + 64)
finally:
    cap.release()
    cv2.destroyAllWindows()


In [22]:
import os, stat, time
from pathlib import Path
import cv2
from yt_dlp import YoutubeDL
import imageio_ffmpeg
import numpy as np

# --- sanity checks on prerequisites (defined in earlier cells) ---
for name in ["sess","inp_name","preprocess","postprocess","draw_dets","IMG_SIZE","CONF_THR","IOU_THR"]:
    assert name in globals(), f"Missing {name}. Run the earlier ONNX cells first."

VIDEO_URL = "https://www.youtube.com/watch?v=-D5lO5Pl-3Q"  # replace if needed
TARGET_H  = 720  # 480/720 is easier to decode fast on CPU

# Ensure ffmpeg is available on PATH
ffmpeg_path = imageio_ffmpeg.get_ffmpeg_exe()
try: os.chmod(ffmpeg_path, os.stat(ffmpeg_path).st_mode | stat.S_IEXEC)
except Exception: pass
os.environ["PATH"] = os.path.dirname(ffmpeg_path) + os.pathsep + os.environ.get("PATH","")

def download_mp4_h264(url: str, max_h=720) -> Path:
    """
    Always download a progressive mp4 (H.264 video + m4a audio).
    This avoids brittle live-stream quirks and AV1 decode issues.
    """
    out_dir = Path("downloads"); out_dir.mkdir(parents=True, exist_ok=True)
    ydl_opts = {
        "outtmpl": str(out_dir / "%(title).90s.%(ext)s"),
        "format": (
            f"bestvideo[ext=mp4][vcodec^=avc1][height<={max_h}]+bestaudio[ext=m4a]/"
            f"best[ext=mp4][vcodec^=avc1][height<={max_h}]/best"
        ),
        "merge_output_format": "mp4",
        "ffmpeg_location": os.path.dirname(ffmpeg_path),
        "noplaylist": True,
        "quiet": True,
    }
    with YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(url, download=True)
        fn = Path(ydl.prepare_filename(info)).with_suffix(".mp4")
    assert fn.exists(), "yt-dlp did not produce an MP4"
    return fn

def open_and_verify_cap(video_path: Path) -> cv2.VideoCapture:
    """
    Open the file and make sure we can read at least one frame.
    Also prints resolution/FPS, and dumps the first frame to disk for debugging.
    """
    cap = cv2.VideoCapture(str(video_path), cv2.CAP_FFMPEG)
    if not cap.isOpened():
        raise RuntimeError(f"Could not open: {video_path}")

    # Try to read a first frame (with small retry loop)
    ok, frame = cap.read()
    tries = 0
    while (not ok or frame is None) and tries < 50:
        ok, frame = cap.read()
        tries += 1

    if not ok or frame is None:
        cap.release()
        raise RuntimeError("Video opened but returned no frames (codec issue?).")

    h, w = frame.shape[:2]
    fps = cap.get(cv2.CAP_PROP_FPS) or 25.0
    print(f"✅ First frame OK: {w}x{h} @ ~{fps:.1f} FPS")
    dbg = Path("downloads/first_frame_debug.jpg")
    cv2.imwrite(str(dbg), frame)
    print("First frame saved:", dbg)

    # Reset to the first frame for playback
    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
    return cap

# 1) Download progressive MP4
mp4 = download_mp4_h264(VIDEO_URL, max_h=TARGET_H)
print("Using MP4:", mp4)

# 2) Open and verify frames
cap = open_and_verify_cap(mp4)

# 3) Play with inference
cv2.namedWindow("ONNX YOLOv8 — Video (q quit, -/= conf, [/] size)", cv2.WINDOW_NORMAL)
cv2.resizeWindow("ONNX YOLOv8 — Video (q quit, -/= conf, [/] size)", 960, 540)

t0, shown = time.time(), 0
try:
    while True:
        ok, frame = cap.read()
        if not ok:
            break

        x, r, padw, padh, Wc, Hc = preprocess(frame)
        outs = sess.run(None, {inp_name: x})
        dets = postprocess(outs, r, padw, padh, frame.shape[1], frame.shape[0], CONF_THR, IOU_THR)
        out  = draw_dets(frame.copy(), dets)

        shown += 1
        if shown % 10 == 0:
            fps = shown / (time.time() - t0 + 1e-9)
            cv2.putText(out, f"Disp~{fps:.1f} conf:{CONF_THR:.2f} imgsz:{IMG_SIZE}",
                        (10,28), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2, cv2.LINE_AA)

        cv2.imshow("ONNX YOLOv8 — Video (q quit, -/= conf, [/] size)", out)
        k = cv2.waitKey(1) & 0xFF
        if k == ord('q'): break
        elif k == ord('-'): CONF_THR = max(0.01, CONF_THR - 0.05)
        elif k in (ord('='), ord('+')): CONF_THR = min(0.99, CONF_THR + 0.05)
        elif k == ord('['): IMG_SIZE = max(320, IMG_SIZE - 64)
        elif k == ord(']'): IMG_SIZE = min(1536, IMG_SIZE + 64)

finally:
    cap.release()
    cv2.destroyAllWindows()


Using MP4: downloads/FC Dallas vs. LAFC ｜ Full Match Highlights ｜ Son Heung-Min BANGER!!.mp4
✅ First frame OK: 1920x1080 @ ~59.9 FPS
First frame saved: downloads/first_frame_debug.jpg


In [10]:
import os, time, shutil, random, subprocess, stat, json
from pathlib import Path
from typing import Optional, List
import yaml, onnx, onnxruntime as ort
import numpy as np

# Small helper: find an executable on PATH
def which(cands: List[str]) -> Optional[str]:
    for c in cands:
        for d in os.environ.get("PATH","").split(os.pathsep):
            p = Path(d) / c
            if p.exists() and os.access(p, os.X_OK):
                return str(p)
    return None

# Keep all artifacts together
WORK_DIR   = Path("hailo_build")
EXPORTS    = WORK_DIR / "exports"
CALIB_DIR  = WORK_DIR / "calib"
HEF_DIR    = WORK_DIR / "hef"
REPORT_DIR = WORK_DIR / "report"
for d in (EXPORTS, CALIB_DIR, HEF_DIR, REPORT_DIR):
    d.mkdir(parents=True, exist_ok=True)


In [12]:
# Your trained run + dataset root
RUN_DIR       = Path("runs/train/soccer_players_clean-exp2")
MODEL_PT      = RUN_DIR / "weights/best.pt"
DATASET_ROOT  = Path("/home/surya/Downloads/soccer players.v1i.yolov8")
IMG_SIZE      = 960         # must match your export
CALIB_MAX     = 800         # images for INT8 calibration

# Use the latest ONNX produced earlier
onnx_list = sorted(EXPORTS.glob("*.onnx"), key=lambda p: p.stat().st_mtime, reverse=True)
assert onnx_list, "No ONNX in hailo_build/exports — run the ONNX export cell first."
ONNX_PATH = onnx_list[0]
print("Using ONNX:", ONNX_PATH)

# Parse ONNX I/O names (needed by Hailo)
m = onnx.load(str(ONNX_PATH)); onnx.checker.check_model(m)
inputs  = list(m.graph.input)
outputs = list(m.graph.output)
ONNX_INPUT_NAME = inputs[0].name
ONNX_OUTPUTS    = [o.name for o in outputs]
print("Input:", ONNX_INPUT_NAME)
print("Outputs:", ONNX_OUTPUTS)

# Find class names
def find_data_yaml(root: Path):
    if (root / "data.yaml").exists(): return root / "data.yaml"
    for q in root.rglob("data.yaml"):
        return q
    if (RUN_DIR / "data.yaml").exists(): return RUN_DIR / "data.yaml"
    for q in RUN_DIR.rglob("data.yaml"):
        return q
    return None

DATA_YAML = find_data_yaml(DATASET_ROOT)
assert DATA_YAML and DATA_YAML.exists(), f"data.yaml not found under {DATASET_ROOT} or run dir"
spec = yaml.safe_load(DATA_YAML.read_text())
if isinstance(spec.get("names"), dict):
    CLASS_NAMES = [spec["names"][i] for i in sorted(map(int, spec["names"].keys()))]
else:
    CLASS_NAMES = list(spec.get("names", []))
NUM_CLASSES = len(CLASS_NAMES) if CLASS_NAMES else spec.get("nc", 80)
print("Classes:", CLASS_NAMES)
print("num_classes:", NUM_CLASSES)


Using ONNX: hailo_build/exports/best.onnx
Input: images
Outputs: ['output0']
Classes: ['0', '1', '2']
num_classes: 3


In [13]:
# Pick up to CALIB_MAX images from dataset (train/val/test folders)
EXTS = {".jpg",".jpeg",".png",".bmp",".tif",".tiff"}
cands = [p for p in DATASET_ROOT.rglob("*") if p.suffix.lower() in EXTS]
random.seed(42); random.shuffle(cands)
pick = cands[:CALIB_MAX] if len(cands) > CALIB_MAX else cands

copied = 0
for p in pick:
    dst = CALIB_DIR / p.name
    if not dst.exists():
        try:
            shutil.copy2(p, dst)
            copied += 1
        except Exception:
            pass
print(f"✅ Calibration images prepared: {len(list(CALIB_DIR.glob('*')))} at {CALIB_DIR.resolve()}")


✅ Calibration images prepared: 163 at /home/surya/c/hailo_build/calib


In [14]:
DFC_YAML = WORK_DIR / "model.yaml"
text = f"""network:
  name: yolo_v8_custom
  inputs:
    - name: {ONNX_INPUT_NAME}
      shape: [1, 3, {IMG_SIZE}, {IMG_SIZE}]
      format: NCHW
      normalization:
        mean: [0.0, 0.0, 0.0]
        std:  [1.0, 1.0, 1.0]
        rescale: 1.0
  outputs:
"""
for n in ONNX_OUTPUTS:
    text += f"    - name: {n}\n"

text += f"""
postprocess:
  type: yolo_v8
  num_classes: {NUM_CLASSES}
  iou_threshold: 0.65
  score_threshold: 0.25
  input_shape: [{IMG_SIZE}, {IMG_SIZE}]

quantization:
  scheme: symmetric
  per_channel: true
  calibration:
    images_dir: {CALIB_DIR.as_posix()}
    max_images: {CALIB_MAX}

compiler:
  target_device: hailo8l
  output_dir: {HEF_DIR.as_posix()}
  output_name: model
"""
DFC_YAML.write_text(text)
print("📝 Wrote DFC YAML:", DFC_YAML.resolve())
print(DFC_YAML.read_text()[:500], "...\n")


📝 Wrote DFC YAML: /home/surya/c/hailo_build/model.yaml
network:
  name: yolo_v8_custom
  inputs:
    - name: images
      shape: [1, 3, 960, 960]
      format: NCHW
      normalization:
        mean: [0.0, 0.0, 0.0]
        std:  [1.0, 1.0, 1.0]
        rescale: 1.0
  outputs:
    - name: output0

postprocess:
  type: yolo_v8
  num_classes: 3
  iou_threshold: 0.65
  score_threshold: 0.25
  input_shape: [960, 960]

quantization:
  scheme: symmetric
  per_channel: true
  calibration:
    images_dir: hailo_build/calib
    max_images: 800

compiler:
  t ...



In [15]:
from pathlib import Path
ONNX_PATH = Path("hailo_build/exports/best.onnx")  # or whatever you used
assert ONNX_PATH.exists() and ONNX_PATH.stat().st_size > 1_000_000, "ONNX missing or too small"
print("OK: file exists →", ONNX_PATH, "size:", round(ONNX_PATH.stat().st_size/1e6, 2), "MB")


OK: file exists → hailo_build/exports/best.onnx size: 12.45 MB


In [8]:
import onnx
m = onnx.load(str(ONNX_PATH))
onnx.checker.check_model(m)
print("OK: ONNX structure is valid")


OK: ONNX structure is valid


In [9]:
def shape_str(v):
    return [d.dim_value if d.HasField("dim_value") else "?" for d in v.type.tensor_type.shape.dim]

inputs  = list(m.graph.input)
outputs = list(m.graph.output)

print("Inputs:")
for i in inputs:  print(" -", i.name, shape_str(i))
print("Outputs:")
for o in outputs: print(" -", o.name, shape_str(o))


Inputs:
 - images [1, 3, 960, 960]
Outputs:
 - output0 [1, 7, 18900]


In [10]:
import onnxruntime as ort, numpy as np

providers = ["CPUExecutionProvider"]
if "CUDAExecutionProvider" in ort.get_available_providers():
    providers = ["CUDAExecutionProvider","CPUExecutionProvider"]

sess = ort.InferenceSession(str(ONNX_PATH), providers=providers)
inp_name = sess.get_inputs()[0].name

IMG_SIZE = 960  # match your export size
x = np.random.rand(1,3,IMG_SIZE,IMG_SIZE).astype(np.float32)
outs = sess.run(None, {inp_name: x})
print("OK: ORT ran. Output shapes:", [o.shape for o in outs])


OK: ORT ran. Output shapes: [(1, 7, 18900)]


In [11]:
from collections import Counter
ops = Counter(n.op_type for n in m.graph.node)
print({k: ops[k] for k in ["QuantizeLinear","DequantizeLinear","QLinearConv","Conv"]})
# If Quantize*/QLinear* > 0 → quantized ONNX; otherwise float.


{'QuantizeLinear': 0, 'DequantizeLinear': 0, 'QLinearConv': 0, 'Conv': 64}


In [16]:
from pathlib import Path
import yaml, onnx

# ==== EDIT THESE ====
DATASET_ROOT = Path("/home/surya/Downloads/soccer players.v1i.yolov8")
RUN_DIR      = Path("runs/train/soccer_players_clean-exp2")
IMG_SIZE     = 960          # must match your ONNX export size
# =====================

WORK_DIR   = Path("hailo_build")
EXPORTS    = WORK_DIR / "exports"
CALIB_DIR  = WORK_DIR / "calib"
HEF_DIR    = WORK_DIR / "hef"
for d in (EXPORTS, CALIB_DIR, HEF_DIR): d.mkdir(parents=True, exist_ok=True)

# Prefer float ONNX (best.onnx) for Hailo; fall back to latest .onnx
cands = [p for p in EXPORTS.glob("*.onnx")]
assert cands, "No ONNX found in hailo_build/exports — export first."
prefer = [p for p in cands if "best.onnx" in p.name.lower()]
ONNX_PATH = prefer[0] if prefer else sorted(cands, key=lambda p: p.stat().st_mtime, reverse=True)[0]
print("Using ONNX:", ONNX_PATH)

# ONNX I/O names (needed for DFC YAML)
m = onnx.load(str(ONNX_PATH)); onnx.checker.check_model(m)
inputs  = list(m.graph.input);  assert inputs, "ONNX has no inputs?"
outputs = list(m.graph.output); assert outputs, "ONNX has no outputs?"
ONNX_INPUT_NAME = inputs[0].name
ONNX_OUTPUTS    = [o.name for o in outputs]
print("Input:", ONNX_INPUT_NAME)
print("Outputs:", ONNX_OUTPUTS)

# Get class names from data.yaml (roboflow export places it at the root)
def find_data_yaml(root: Path):
    if (root / "data.yaml").exists(): return root / "data.yaml"
    for q in root.rglob("data.yaml"): return q
    return None

DATA_YAML = find_data_yaml(DATASET_ROOT)
assert DATA_YAML and DATA_YAML.exists(), f"data.yaml not found under {DATASET_ROOT}"
spec = yaml.safe_load(DATA_YAML.read_text())
names = spec.get("names", [])
if isinstance(names, dict):
    CLASS_NAMES = [names[i] for i in sorted(map(int, names.keys()))]
elif isinstance(names, list):
    CLASS_NAMES = names
else:
    CLASS_NAMES = []
NUM_CLASSES = len(CLASS_NAMES) if CLASS_NAMES else spec.get("nc", 80)
print("num_classes:", NUM_CLASSES, "| classes:", CLASS_NAMES)


Using ONNX: hailo_build/exports/best.onnx
Input: images
Outputs: ['output0']
num_classes: 3 | classes: ['0', '1', '2']


In [17]:
import shutil, random

CALIB_MAX = 800  # 400–1000 is typical
exts = {".jpg",".jpeg",".png",".bmp",".tif",".tiff"}

cands = [p for p in DATASET_ROOT.rglob("*") if p.suffix.lower() in exts]
random.seed(42); random.shuffle(cands)
pick = cands[:CALIB_MAX] if len(cands) > CALIB_MAX else cands

copied = 0
for p in pick:
    dst = CALIB_DIR / p.name
    if not dst.exists():
        try:
            shutil.copy2(p, dst); copied += 1
        except Exception:
            pass

print(f"✅ Calibration images prepared: {len(list(CALIB_DIR.glob('*')))} at {CALIB_DIR.resolve()}")


✅ Calibration images prepared: 163 at /home/surya/c/hailo_build/calib


In [ ]:
DFC_YAML = WORK_DIR / "model.yaml"

text = f"""network:
  name: yolo_v8_custom
  inputs:
    - name: {ONNX_INPUT_NAME}
      shape: [1, 3, {IMG_SIZE}, {IMG_SIZE}]
      format: NCHW
      normalization:
        mean: [0.0, 0.0, 0.0]
        std:  [1.0, 1.0, 1.0]
        rescale: 1.0
  outputs:
"""
for n in ONNX_OUTPUTS:
    text += f"    - name: {n}\n"

text += f"""
postprocess:
  type: yolo_v8
  num_classes: {NUM_CLASSES}
  iou_threshold: 0.65
  score_threshold: 0.25
  input_shape: [{IMG_SIZE}, {IMG_SIZE}]

quantization:
  scheme: symmetric
  per_channel: true
  calibration:
    images_dir: {CALIB_DIR.as_posix()}
    max_images: {CALIB_MAX}

compiler:
  target_device: hailo8l
  output_dir: {HEF_DIR.as_posix()}
  output_name: model
"""


DFC_YAML.write_text(text)
print("📝 Wrote DFC YAML:", DFC_YAML.resolve())
print("--- preview ---\n", DFC_YAML.read_text()[:600], "\n--- end ---")


📝 Wrote DFC YAML: /home/surya/c/hailo_build/model.yaml
--- preview ---
 network:
  name: yolo_v8_custom
  inputs:
    - name: images
      shape: [1, 3, 960, 960]
      format: NCHW
      normalization:
        mean: [0.0, 0.0, 0.0]
        std:  [1.0, 1.0, 1.0]
        rescale: 1.0
  outputs:
    - name: output0

postprocess:
  type: yolo_v8
  num_classes: 3
  iou_threshold: 0.65
  score_threshold: 0.25
  input_shape: [960, 960]

quantization:
  scheme: symmetric
  per_channel: true
  calibration:
    images_dir: hailo_build/calib
    max_images: 800

compiler:
  target_device: hailo8l
  output_dir: hailo_build/hef
  output_name: model
 
--- end ---


In [1]:
def channel_from_shape(shp):
    # shp is like [B, C, N] for your model
    if len(shp) == 3:   # [1, C, N]
        return shp[1]
    if len(shp) == 4:   # [1, C, H, W]
        return shp[1]
    return None


In [2]:
from pathlib import Path
import yaml, onnx
from onnx import shape_inference
import onnxruntime as ort
import numpy as np

# ==== EDIT THESE ====
ONNX_PATH = Path("hailo_build/exports/best.onnx")   # your best.onnx path
DATA_YAML = Path("/home/surya/Downloads/soccer players.v1i.yolov8/data.yaml")
IMG_SIZE  = 960
# ====================

def dims_list(v):
    out = []
    for d in v.type.tensor_type.shape.dim:
        if d.HasField("dim_value"): out.append(int(d.dim_value))
        elif d.HasField("dim_param") and d.dim_param: out.append(d.dim_param)
        else: out.append("?")
    return out

# Load classes
spec = yaml.safe_load(DATA_YAML.read_text())
names = spec.get("names", [])
if isinstance(names, dict):
    CLASS_NAMES = [names[i] for i in sorted(map(int, names.keys()))]
elif isinstance(names, list):
    CLASS_NAMES = names
else:
    CLASS_NAMES = []
NC = len(CLASS_NAMES) if CLASS_NAMES else int(spec.get("nc", 80))
print("Classes:", CLASS_NAMES)
print("num_classes (NC):", NC)

# Load ONNX & infer shapes
m = onnx.load(str(ONNX_PATH))
onnx.checker.check_model(m)
try:
    mi = shape_inference.infer_shapes(m)
except Exception:
    mi = m

inputs  = list(mi.graph.input)
outputs = list(mi.graph.output)
print("\nInputs:")
for i in inputs:  print(" -", i.name, dims_list(i))
print("Outputs:")
for o in outputs: print(" -", o.name, dims_list(o))

# Robust head checker: accepts [B,C,N] or [B,N,C] or [B,C,H,W]
def check_head_shape(shape, nc):
    # returns (ok:bool, layout:str, C:int|None, N:int|None, msg:str)
    if len(shape) == 4:
        b,c,h,w = shape
        if isinstance(c, int) and c in (4+nc, 5+nc):
            return True, "BCHW", c, (h,w), f"C={c} in {{4+{nc},5+{nc}}}"
        return False, "BCHW", c if isinstance(c,int) else None, (h,w), f"C={c} not in {{4+{nc},5+{nc}}}"

    if len(shape) == 3:
        b,d1,d2 = shape
        cand = []
        for ax, dim in enumerate([d1,d2], start=1):  # axes 1 or 2 can be C
            if isinstance(dim,int) and dim in (4+nc, 5+nc):
                layout = "BCN" if ax==1 else "BNC"
                C = dim
                N = d2 if ax==1 else d1
                return True, layout, C, N, f"channels on axis {ax}: C={C} in {{4+{nc},5+{nc}}}"
        return False, "B?N", None, None, f"no axis equals 4+{nc} or 5+{nc} (got {d1},{d2})"

    return False, "other", None, None, f"unsupported rank: {len(shape)}"

# Static checks
any_ok = False
for o in outputs:
    shp = dims_list(o)
    ok, layout, C, N, msg = check_head_shape(shp, NC)
    any_ok |= ok
    print(f"\n[o={o.name}] shape={shp} → {layout} | {msg}")

# If shape ranks/values were unknown, confirm with ORT
if not any_ok or any("?" in map(str,dims_list(o)) for o in outputs):
    sess = ort.InferenceSession(str(ONNX_PATH), providers=["CPUExecutionProvider"])
    inp  = sess.get_inputs()[0].name
    x = np.random.rand(1,3,IMG_SIZE,IMG_SIZE).astype(np.float32)
    outs = sess.run(None, {inp: x})
    print("\nRuntime output shapes:", [list(a.shape) for a in outs])
    # Re-check using runtime shapes
    any_ok_rt = False
    for a in outs:
        ok, layout, C, N, msg = check_head_shape(list(a.shape), NC)
        any_ok_rt |= ok
        print(" - runtime:", list(a.shape), "→", layout, "|", msg)
    any_ok = any_ok or any_ok_rt

print("\nFINAL VERDICT:", "MATCH " if any_ok else "MISMATCH ")


Classes: ['0', '1', '2']
num_classes (NC): 3

Inputs:
 - images [1, 3, 960, 960]
Outputs:
 - output0 [1, 7, 18900]

[o=output0] shape=[1, 7, 18900] → BCN | channels on axis 1: C=7 in {4+3,5+3}

FINAL VERDICT: MATCH 


In [3]:
from ultralytics import YOLO

# Load YOLOv8 model
model = YOLO("yolov8n.pt")

# Print model summary
model.info()

# Visualize model (requires graphviz installed)
model.info()


YOLOv8n summary: 225 layers, 3157200 parameters, 0 gradients, 8.9 GFLOPs


/home/surya/c/.venv/lib/python3.12/site-packages/ultralytics/nn/tasks.py:732: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(file, map_location="cpu")


YOLOv8n summary: 225 layers, 3157200 parameters, 0 gradients, 8.9 GFLOPs


(225, 3157200, 0, 8.8575488)